# Исследование состояния авторынка Казахстана

Заказчик - компания Меркур Авто - ставит задачу:
- Определить рыночные сегменты на авторынке Казахстанв
- Разработать маркетинговый план развития компании

Нам предоставлены файлы, содержащие в себе данные за 2019 год:
- Сведения о продажи автомобилей в Казахстане 2019 год в разрезе месяцев, моделей автомобилей и их комплектаций и года выпуска автомобилей.

**В исследовании:**
- Произведем автоматизированную загрузку в рабочую папку предоставленных таблиц
- Выясним, по какому признаку возможно объединение
- Для каждого принятого признака произведем:
 - загрузку файлов в тетрадку Юпитера или Коллаба
 - обработку данных
 - объединение файлов и получение сводного датафрейма
 - выгрузку сводных данных в рабочую папку в формате Excel
- Проведем анализ полученных сводных данных.

**Ход исследования:**
- Загрузка данных из предоставленных файлов и подготовка их к анализу
- Изучение общей информации о датафрейме и предобработка данных
- Исследовательский анализ данных:
 - Анализ рынка автомобилей Казахстана
 - Анализ позиции Меркур Авто: внутренная и внешняя среда
- Выводы и рекомендации



## Загрузка и подготовка данных к анализу


**Путь к файлу**

[Ссылка на данные](https://disk.yandex.ru/d/-ehmXsTcOM3rUQ).



Изучим данные и выполним предобработку. Есть ли в данных пропуски и дубликаты? Убедимся, что типы данных во всех колонках соответствуют сохранённым в них значениям.

Загрузим библиотеку Яндекс.Диск

In [ ]:
pip install yadisk --quiet

Note: you may need to restart the kernel to use updated packages.


### Импортируем библиотеки:
- **pandas**  
- **sys**
- **IPython.display** для отображения данных
- **warnings** для отключения предупреждений
- **yadisk** для работы с данными Яндекс.Диска
- **os** для управления папками с данными
- **tqdm** для корректного отображения данных
- **openpyxl** для работы с таблицами Excel
- **logging** - для отображения логов
- **tqdm** - для отображения прцесса вычисления кода
- **re** - для обрабоьки регулярных выражений
- **math** - для округления данных float

In [ ]:
# импортируем библиотеки
import pandas as pd
import sys
from IPython.display import display
import os
from tqdm import tqdm
import warnings
from openpyxl import load_workbook
from openpyxl.cell.text import Font, InlineFont
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles.numbers import BUILTIN_FORMATS
import yadisk
import logging
import re
import math

Перейдем в рабочую область

In [ ]:
os.chdir('/home/jovyan/work')

Введем идентификационные данные для подключения

In [ ]:
from cfg import app_id, secret_id, ya_token

Получим токен для доступа к Яндекс.диску

In [ ]:
# получаем токен для доступа к яндекс диску


y = yadisk.YaDisk(app_id, secret_id, ya_token)

if y.check_token():
    print("Sucessfully received token!")
else:
    print("Something went wrong. Not sure how though...")


Sucessfully received token!


Создадим список файлов для загрузки с Яндекс.Диска

In [ ]:
# создаем список файлов для загрузки
list_of_files = []
for el in list(y.listdir('car_market_of_Kazakhstan')):
    if el['path'].endswith('.csv'):
        list_of_files.append(el['path'])
print("\033[34m""\033[1m""Создан список файлов list_of_files")
list_of_files

Создан список файлов list_of_files


['disk:/car_market_of_Kazakhstan/auto_kz_2019_датасет.csv']

### Для отслеживания работы кода создадим файл `mylog.log`, куда будем сохранять логи.   


In [ ]:
logging.basicConfig(
    level=logging.INFO,
    filename = "mylog_cars.log",
    format = "%(asctime)s - %(module)s - %(levelname)s - %(funcName)s: %(lineno)d - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    filemode="w"
    )

logging.info('info')
logging.debug('debug')
logging.warning('warning_message')
logging.error("An ERROR")
logging.critical("A CRITICAL")

### Создадим функции

Создадим функцию `load_files_collab` для загрузки данных в Гугл Коллаб

In [ ]:
def load_files_collab(url_path, file_name):
      Da = gc.open_by_url(url_path)
      Da_df = Da.worksheet(file_name)
      rows = Da_df.get_all_values()
      df= pd.DataFrame.from_records(rows[1:], columns = rows[0])
      #display(df)
      print("\033[34m""\033[1m""Первые строки датафрейма")
      display(df.head(5))
      print("\033[34m""\033[1m""Информация о типах данных")
      print()
      display(df.info())
      print("\033[34m""\033[1m""Доля пропущенных значений ")
      display(round(df.isna().mean() * 100).astype(int))
      return(df)

Создадим функцию `load_files` для загрузки данных в Юпитер

In [ ]:
def load_files(file_name):
    df= pd.read_csv(file_name, sep = ';')
    print("\033[34m""\033[1m""Первые строки датафрейма")
    display(df.head(5))
    print("\033[34m""\033[1m""Информация о типах данных")
    print()
    display(df.info())
    print("\033[34m""\033[1m""Доля пропущенных значений ")
    display(round(df.isna().mean() * 100).astype(int))
    print("\033[34m""\033[1m""Количество явных дубликатов ")
    display(df.duplicated().sum())
    return(df)

Создадим функцию `go_to_numeric` для приведения данных к числовому формату

In [ ]:
def go_to_numeric(column_name):
    data[column_name] = pd.to_numeric(data[column_name])

Функцию `df_download`для выгрузки файлов с наиболее характерными параметрами и целесообразными результатами в форматах:
- .csv
- .xlsx

In [ ]:
def df_download(data_name, df_name_xlsx, df_name_csv):

    # Выгрузим файл в формате .csv
    data_name.to_csv(df_name_csv)
    print("\033[34m"'Выгружен файл '"\033[1m", df_name_csv, "\033[0m")

    # Выгрузим файл в папку в формате Excel
    # Воспользуемся функцией to_excel() , для чего ранее мы установили openpyxl, чтобы записывать файлы в Excel.

    workbook = Workbook()
    sheet = workbook.active

    for row in dataframe_to_rows(data_name, header=True):
        sheet.append(row)

    # изменим высоту первой строки
    sheet.row_dimensions[1].height = 30

    # изменим ширину некоторых колонок
    columns_to_change = [chr(i) for i in range(66, 81)]
    for cell in columns_to_change:
        sheet.column_dimensions[cell].width = 12

    # изменим ширину некоторых колонок
    columns_to_change_2 = [chr(i) for i in range(87, 89)] + [chr(i) for i in range(81, 83)]
    for cell in columns_to_change_2:
        sheet.column_dimensions[cell].width = 20

    # сохраним заданную изначально размерность колонок с результатами
    columns_to_save_dim = ['L']
    for cell in columns_to_save_dim:
        for row in range(2, data_name.shape[0]+3):
            row=str(row)
            sheet[cell+row].number_format = '0.0'

    # зададим размерность колонок с датой
    columns_to_save_dim_time = ['D']
    for cell in columns_to_save_dim_time:
        for row in range(2, data_name.shape[0]+3):
            row=str(row)
            sheet[cell+row].number_format = 'dd.mm.yyyy'

    # установим жирный шрифт для ячеек первой колонки
    columns_to_font = ['E']
    for cell in columns_to_font:
        for row in range(2, data_name.shape[0]+3):
            row=str(row)
            sheet[cell+row].font = Font(size=12, bold=True)

    # установим жирный шрифт для ячеек первой строки
    cell_to_change = ['A1','B1','C1','D1','E1','F1','G1','H1','I1','J1','G1','K1','L1','M1','N1','O1'
                     ,'P1','Q1','R1','S1','T1','U1','V1','W1','X1']
    for cell in cell_to_change:
        sheet[cell].font = Font(size=12, bold=True)

    # удалим 2 строку (пустую)
    sheet.delete_rows(2)

    # зафиксируем первую строку при прокрутке по вертикали
    sheet.freeze_panes = 'A2'

    # выгрузим таблицу в рабочую область
    workbook.save(df_name_xlsx)
    print("\033[34m"'Выгружен файл '"\033[1m", df_name_xlsx, "\033[0m")

    return

### Загрузка файлов в рабочую папку

Создадим в рабочей области папку для загрузки файлов с Яндекс.Диска и перейдем в нее

In [ ]:
# создаем папку загрузки
load_path = 'car_market_files'
if not os.path.exists(load_path):
    os.mkdir(load_path)
os.chdir(load_path)

Загрузим (скопируем) файлы с Яндекс.Диска в созданную папку в рабочей области

In [ ]:
# загружаем файлы с яндекс диска
for file in tqdm(list_of_files):
    y.download(file.split(':')[1], file.split('/')[-1])
    #y.download()
print("\033[34m""\033[1m""Файлы загружены в папку в рабочей области car_market_files")

100%|██████████| 1/1 [00:02<00:00,  2.03s/it]

Файлы загружены в папку в рабочей области car_market_files


Перейдем в рабочую папку

In [ ]:
os.chdir(path="/home/jovyan/work/car_market_files")

### Загрузка и изучение данных

Загрузим в тетрадку файл. Воспользуемся функцией `load_files`  
Заказчик сообщил нам структуру файла

|Название столбца |Что означает |
|:-----|:----|
Год | год продажи (2019)
Месяц | месяц продажи (январь - сентябрь)
Компания | название автоцентра
Бренд | название продаваемой марки автомобиля
Модель | название модели автомобиля
Модификация | модификация модели автомобиля (удаляем)
Год выпуска | год производства автомобиля
Страна-производитель | страна, где произведен автомобиль
Вид топлива | бензин, дизель, электричество, гибрид
Объём двигателя л | объем двигателя автомобиля в литрах
Коробка передач | тип коробки переключения передач (оставляем два варианта: автоматическая, механическая, то есть все что не механика ставим автомат, на DSG, S-Tronic и прочее делить не надо, равно как и количество передач)
Тип привода | в итоге оставляем RWD - задний привод, FWD - передний привод, 4WD - полный привод, 2WD - все остальное (подключаемый полный привод и где нет четкого указания передний или задний это привод)
Сегмент | сегмент, к которому относится авто (удаляем)
Регион | регион продажи
Наименование дилерского центра | совпадает с компанией - можно удалить
Тип клиента | юридическое или физическое лицо (в рамках анализа не критично - можно удалить)
Форма расчета | наличный и безналичный расчет (много пропусков - можно удалять)
Количество | количество автомобилей в заказе
Цена USD | цена автомобиля
Продажа USD | цена заказа (цена авто умноженная на количество и за вычетом скидок если есть)
Область | область продажи
Сегментация 2013 | сегмент автомобиля актуальный
Класс 2013 | класс автомобиля актуальный
Сегментация Eng | английская сегментация (удаляем)
Локализация производства | удаляем (совпадает со страной производителем)
Признаки-категории:
 |- сегмент
 |- класс
 |- тип привода
 |- коробка передач


Определим, что будут отображаться все имеющиеся в датафрейме столбцы

In [ ]:
pd.set_option('max_columns', None)

Загрузим файл из рабочей папки и ознакомимся с ним

In [ ]:
try:
    data = load_files_collab('https://docs.google.com/spreadsheets/d/1mvrRlLTL3U6Gu1HJsPM-b0MhvF6r8uF2MZu4E9NqQgk/edit?usp=sharing', 'innopolistg')
except:
    data = load_files(os.listdir()[0])

Первые строки датафрейма


,Год,Месяц,Компания,Бренд,Модель,Модификация,Год выпуска,Страна-производитель,Вид топлива,"Объём двиг, л,",Коробка передач,Тип привода,Сегмент,Регион,Наименование дилерского центра,Тип клиента,Форма расчета,Количество,"Цена, USD","Продажа, USD",Область,Сегментация 2013,Класс 2013,Сегментация Eng,Локализация производства
0,2019,Май,Mercur Auto,Audi,A3,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,безналичный,1.0,28115,28115,г.Алматы,Легковые автомобили,C класс,C,Импорт
1,2019,Август,Mercur Auto,Audi,A3,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Юр. Лицо,наличный,1.0,"32246,99","32246,99",г.Алматы,Легковые автомобили,C класс,C,Импорт
2,2019,Апрель,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-Tronic,FWD,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,безналичный,1.0,32000,32000,г.Алматы,Легковые автомобили,D класс,D,Импорт
3,2019,Июль,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Юр. Лицо,безналичный,1.0,31929,31929,г.Алматы,Легковые автомобили,D класс,D,Импорт
4,2019,Июль,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,наличный,1.0,31929,31929,г.Алматы,Легковые автомобили,D класс,D,Импорт


Информация о типах данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39966 entries, 0 to 39965
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Год                             39966 non-null  int64  
 1   Месяц                           39966 non-null  object 
 2   Компания                        39966 non-null  object 
 3   Бренд                           39966 non-null  object 
 4   Модель                          39966 non-null  object 
 5   Модификация                     36375 non-null  object 
 6   Год выпуска                     39465 non-null  object 
 7   Страна-производитель            39966 non-null  object 
 8   Вид топлива                     36826 non-null  object 
 9   Объём двиг, л,                  35708 non-null  object 
 10  Коробка передач                 36711 non-null  object 
 11  Тип привода                     35677 non-null  object 
 12  Сегме

None

Доля пропущенных значений 


Год                                0
Месяц                              0
Компания                           0
Бренд                              0
Модель                             0
Модификация                        9
Год выпуска                        1
Страна-производитель               0
Вид топлива                        8
Объём двиг, л,                    11
Коробка передач                    8
Тип привода                       11
Сегмент                           17
Регион                             0
Наименование дилерского центра     0
Тип клиента                       18
Форма расчета                     65
Количество                         0
Цена, USD                          0
Продажа, USD                       0
Область                            0
Сегментация 2013                   0
Класс 2013                         0
Сегментация Eng                    0
Локализация производства           0
dtype: int64

Количество явных дубликатов 


18698

In [ ]:
data['Вид топлива'].unique()

array(['Бензин', 'Дизель', nan, 'дизель ', 'бензин ', 'гибрид', '2',
       '1,6', '0', 'Электро', 'Электричество'], dtype=object)

In [ ]:
data.loc[(data['Вид топлива'].isna())].count()

Год                               3140
Месяц                             3140
Компания                          3140
Бренд                             3140
Модель                            3140
Модификация                       2919
Год выпуска                       2721
Страна-производитель              3140
Вид топлива                          0
Объём двиг, л,                     324
Коробка передач                    532
Тип привода                        550
Сегмент                            827
Регион                            3140
Наименование дилерского центра    3140
Тип клиента                       1876
Форма расчета                      468
Количество                        3140
Цена, USD                         3140
Продажа, USD                      3140
Область                           3140
Сегментация 2013                  3140
Класс 2013                        3140
Сегментация Eng                   3140
Локализация производства          3140
dtype: int64

Ознакомимся с рядом качественных показателей

In [ ]:
print("\033[34m""\033[1m"
    ' Количество уникальных Компаний - ', data['Компания'].nunique(),'\n',
    'Количество уникальных Брендов - ', data['Бренд'].nunique(),'\n',
      'Количество уникальных Регионов - ', data['Регион'].nunique(),'\n',
      'Количество уникальных Областей - ', data['Область'].nunique(),'\n',
    )

 Количество уникальных Компаний -  40 
 Количество уникальных Брендов -  46 
 Количество уникальных Регионов -  26 
 Количество уникальных Областей -  18 



In [ ]:
print("\033[34m""\033[1m"
      'Список Регионов - ', data['Регион'].unique(),'\n',
      'Список Областей - ', data['Область'].unique(),'\n')

Список Регионов -  ['Алматы' 'Нур-Султан' 'Усть-Каменогорск' 'Уральск' 'Костанай' 'Павлодар'
 'Семей' 'Шымкент' 'Актау' 'Атырау' 'Актобе' 'Караганда' 'Петропавловск'
 'Кокшетау' 'Рудный' 'Кызылорда' 'Тараз' 'Туркестан' 'Талдыкорган'
 'ЭКСПОРТ' 'Кульсары' 'Каскелен' 'Экибастуз' 'Риддер' 'Сарыагаш'
 'Зыряновск'] 
 Список Областей -  ['г.Алматы' 'г.Нур-Султан' 'Восточно-Казахстанская область'
 'Западно-Казахстанская область' 'Костанайская область'
 'Павлодарская область' 'Южно-Казахстанская область'
 'Мангистауская область' 'Атырауская область' 'Актюбинская область'
 'Карагандинская область' 'Северо-Казахстанская область'
 'Акмолинская область' 'Кызылординская область' 'Жамбылская область'
 'Туркестанская область' 'Алматинская область' 'Экспорт область'] 



<div style="border-radius: 45px;border:solid green 7px; padding: 40px">

Отметим, что:
- общее количество записей - почти 40 тысяч
- названия столбцов представлены НЕ по правилам хорошего тона
- в датафрейме 25 столбцов и часть из них неинформативна, можно удалить    
- почти все данные представлены в формате `object`. Потребуется данные в столбце:
 - `Год выпуска`,	`Объём двиг, л,`,	`Цена, USD`, 	`Продажа, USD` - привести к числовому формату
- в нескольких столбцах еть пропущенные значения
- количество явных дубликатов - более 18 тысяч, то есть почти половина записей продублирована.
    - **По пояснению Заказчика выявленные дубликаты таковыми не являются. Причина здесь в предоставлении неперсонифицированных данных (без указания ID сделки).**
- в датафрейме представлены:
    - Количество уникальных Компаний -  40
    - Количество уникальных Брендов -  46
    - Количество уникальных Регионов -  26
    - Количество уникальных Областей -  18    


<div style="border-radius: 45px;border:solid blue 7px; padding: 40px">

pd.reset_option('max_columns')
    
    для отмены


<a id="4"></a>
## Предварительная обработка данных

### Приведем названия столбцов  к "змеиному" виду

In [ ]:
colms = data.columns.str.lower().str.replace(' ', '_').str.replace(',', '').str.replace('-', '_').tolist()
data.columns = colms

In [ ]:
print(data.columns)

Index(['год', 'месяц', 'компания', 'бренд', 'модель', 'модификация',
       'год_выпуска', 'страна_производитель', 'вид_топлива', 'объём_двиг_л',
       'коробка_передач', 'тип_привода', 'сегмент', 'регион',
       'наименование_дилерского_центра', 'тип_клиента', 'форма_расчета',
       'количество', 'цена_usd', 'продажа_usd', 'область', 'сегментация_2013',
       'класс_2013', 'сегментация_eng', 'локализация_производства'],
      dtype='object')


### Удалим столбцы, которые не окажут влияния на анализ в соответствии с ТЗ.
Заказчик в ТЗ довел до нас, что следующие столбцы не окажут влияния на анализ и их можно удалить:

|Название столбца |Что означает |
|:-----|:----|
- Модификация | модификация модели автомобиля (удаляем)
- Сегмент | сегмент, к которому относится авто (удаляем)
- Наименование дилерского центра | совпадает с компанией - можно удалить
- Тип клиента | юридическое или физическое лицо (в рамках анализа не критично - можно удалить)
- Форма расчета | наличный и безналичный расчет (много пропусков - можно удалять)
- Сегментация Eng | английская сегментация (удаляем)
- Локализация производства | удаляем (совпадает со страной производителем)

Тем не менее, чтобы не потерять информацию, ознакомимся с содержанием некоторых из них:

#### Оставим пока `Тип клиента` и `Наименование дилерского центра` - они могут быть востребованы при анализе

#### Ознакомимся, какие термины присутствуют в столбце `сегмент`, `сегментация_eng` и `класс_2013`

In [ ]:
print(data.сегмент.unique())

['Легковые автомобили Premium' 'Внедорожники Premium' nan
 'Легковые автомобили' 'B класс ' 'Среднеразмерные SUV ' 'Внедорожники '
 'Внедорожники' 'Полноразмерные SUV ' 'Субкомпактные SUV ' 'салон'
 'Легковые автомобили ' 'B' 'B-SUV' 'C-SUV' 'C' 'D-SUV' 'D' 'MPV' 'LCV'
 'городской' 'малотоннажные' '16+1' 'VAN' 'Ambulance' 'Шасси'
 'Крупнотоннажные' 'среднетоннажные'
 'Промтоварный фургон 4600*2200*2200*40' 'Изотермический фургон'
 'Изотермический фургон 4600*2200*2200' 'Промтоварный фургон Т25 с б.д.'
 'борт' 'Промтоварный фургон' 'Промтоварный фургон 4600*2200*2200*25'
 'Бортовая платформа с тентом'
 'Изотермический фургон с холодильным оборудованием'
 'Промтоварный фургон Т40' 'Изотермический фургон 5200*2200*2200*'
 'Борт 5200*2200*400' 'Промтоварный фургон Т40 с б.д.' 'туристический'
 '43+1' 'Легковой автомобиль' 'внедорожник' 'пикап'
 'коммерческий транспорт' 'коммерческие' 'Коммерческие автомобили'
 'Коммерческие и грузовые' 'Компактные SUV ' 'Легковые' 'Компактные SUV'
 'SUV ' '

In [ ]:
print(sorted(data.сегментация_eng.unique()))

['A', 'B', 'C', 'D', 'E', 'F', 'G2', 'K1', 'K2', 'K3', 'K4', 'L1', 'L2', 'M', 'N', 'O']


In [ ]:
print(sorted(data.класс_2013.unique()))

['A класс', 'B класс', 'C класс', 'D класс', 'E класс', 'F класс', 'Pick-ups', 'Большие автобусы', 'Компактвэн', 'Компактные SUV', 'Крупнотоннажные грузовики', 'Малотоннажные грузовики', 'Микроавтобусы', 'Полноразмерные SUV', 'Полноразмерный Минивэн', 'Развозные автомобили', 'Спортивные автомобили', 'Среднеразмерные SUV', 'Среднетоннажные грузовики', 'Средние автобусы', 'Субкомпактные SUV']


Видим, что в столбцах `сегментация_eng` и `класс_2013` информация совпадает.

<div style="border-radius: 45px;border:solid green 7px; padding: 40px">

В столбцах `сегментация_eng` и `класс_2013` информация дублируется. причем в последнем информация более разнообразная, более полная. Удалим столбец `сегментация_eng`.

В столбце `сегмент` информация отчасти дублирует информацию столбца `класс_2013`, но и содержит отличительные зарактеристик6, например: 'Изотермический фургон 5200*2200*2200*', 'Борт 5200*2200*400' 'Промтоварный фургон Т40 с б.д.' 'туристический'. Пока оставим этот столбец.

#### Ознакомимся, какие термины присутствуют в столбце `модификация`

In [ ]:
data['модификация'].unique()

array(['TFSI', 'TFSI Allroad', 'A8L TFSI', ..., 'B3P2M3TX5C', 'V90CC Pro',
       'KAMAZ 58815Z'], dtype=object)

<div style="border-radius: 45px;border:solid green 7px; padding: 40px">

В столбце `модификация` может присутствовать информация и виде двигателя, трансмиссии. Пока оставим этот столбец.

#### Удалим столбцы, которые не окажут влияния на анализ в соответствии с ТЗ:
|Название столбца |Что означает |
|:-----|:----|
- Форма расчета | наличный и безналичный расчет (много пропусков - можно удалять)
- Сегментация Eng | английская сегментация (удаляем)
- Локализация производства | удаляем (совпадает со страной производителем)


In [ ]:
# Номера столбцов под удаление
colms_to_del_num = [-9,-2,-1]

In [ ]:
for i in colms_to_del_num:
    data = data.drop(colms[i], axis=1)
#data.columns

### Приведем к числовому формату столбцы:
-  `год_выпуска`, `объём_двиг_л`, `цена_usd`, `продажа_usd`

(ранее -назывались как `Год выпуска`, `Объём двиг, л,`, `Цена, USD`, `Продажа, USD`)

In [ ]:
# зададим список столбцов
col_list_four = ['год_выпуска', 'объём_двиг_л', 'цена_usd', 'продажа_usd']

In [ ]:
for j in col_list_four:
    data[j] = data[j].str.strip().str.replace(',', '.')
    data[j] = data[j].str.replace(' ', '')



In [ ]:
data['год_выпуска'] = data['год_выпуска'].str.replace('2\xa0018', '2018')
data['год_выпуска'] = data['год_выпуска'].str.replace('2\xa0019', '2019')

In [ ]:
#data['год_выпуска'] = data['год_выпуска'].str.replace(' ', '')

In [ ]:
int_cols = ['цена_usd', 'продажа_usd']
for col in int_cols:
    # downcast='signed' приводит данные к минимально возможному типу числовых данных
    data[col] = pd.to_numeric(data[col], downcast='signed')
print(data[int_cols].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39966 entries, 0 to 39965
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   цена_usd     39966 non-null  float64
 1   продажа_usd  39966 non-null  float64
dtypes: float64(2)
memory usage: 624.6 KB
None


### Выведем на экран уникальные значения, указанные в столбце `бренд`

In [ ]:
sorted(data.бренд.unique().astype(str))

['ANKAI',
 'Audi',
 'BMW',
 'Cadillac',
 'Chevrolet',
 'Daewoo',
 'Dong Feng',
 'Ford',
 'Foton',
 'GAZ',
 'Hino',
 'Hyundai',
 'Hyundai Truck & Bus',
 'Infiniti',
 'Isuzu',
 'Iveco',
 'Jac',
 'Jaguar',
 'KAMAZ',
 'Kia',
 'Lada',
 'Land Rover',
 'Lexus',
 'Lifan',
 'MAN',
 'Mazda',
 'Mercedes-Benz',
 'Mercedes-Benz Trucks',
 'Mini',
 'Mitsubishi',
 'Nefaz',
 'Nissan',
 'Peugeot',
 'Porsche',
 'Ravon',
 'Renault',
 'Scania',
 'Shacman',
 'Skoda',
 'Subaru',
 'Toyota',
 'UAZ',
 'Volkswagen',
 'Volvo',
 'Yutong',
 'Урал']

Дублирующихся названий брендов не выявлено


### Выведем на экран уникальные значения, указанные в столбце `модель`

In [ ]:
sorted(data.модель.unique().astype(str))

['2206',
 '2217',
 '2310',
 '2705',
 '2752',
 '3',
 '3 серия',
 '300',
 '301',
 '3151',
 '3221',
 '32551',
 '3302',
 '3303',
 '3308',
 '3309',
 '3741',
 '3909',
 '3962',
 '4308',
 '4311',
 '43118',
 '4320',
 '4320-1951-40',
 '4320-1951-60',
 '43253',
 '43502',
 '44108',
 '45141',
 '45142',
 '45143',
 '4x4',
 '5 серия',
 '500',
 '5299',
 '53215',
 '53504',
 '53605',
 '54115',
 '5490',
 '5557',
 '58815Z',
 '6',
 '6 серия',
 '65111',
 '65115',
 '65116',
 '65117',
 '6520',
 '65206',
 '65225',
 '66052',
 '6606',
 '7 серия',
 '8 серия',
 '911 Carrera S',
 'A-Class',
 'A3',
 'A4',
 'A6',
 'A7',
 'A8',
 'ASX',
 'Accent',
 'Actros',
 'Almera',
 'Amarok',
 'Arkana',
 'Aveo',
 'BC 095',
 'BC 211 MA',
 'BJ1069VDJEA-F1',
 'BJ3253DMPKB-AD',
 'BS 106 A',
 'BS 106 D',
 'BWC6665GA5',
 'C-Class',
 'C-HR',
 'CLA-Class',
 'CLS-Class',
 'CX-5',
 'CX-9',
 'Cabrio',
 'Caddy Kasten ',
 'Caddy Maxi Kombi',
 'Camaro',
 'Camry',
 'Captiva',
 'Caravelle',
 'Cargo',
 'Cayenne',
 'Cerato',
 'Corolla',
 'Countryman'

Дублирующихся названий брендов не выявлено


### Обработака данных в столбце `вид_топлива`
Выведем на экран уникальные значения, указанные в столбце `вид_топлива`

In [ ]:
sorted(data.вид_топлива.unique().astype(str))

['0',
 '1,6',
 '2',
 'nan',
 'Бензин',
 'Дизель',
 'Электричество',
 'Электро',
 'бензин ',
 'гибрид',
 'дизель ']

"Почистим" данные:
- '0', '1,6', '2'
    - проверим строки, насколько соответствует отображение данных по столбцам
    - найдем марку и модель автомобиля и найдем в интернете соответствующий вид топлива
- nan -   найдем марку и модель автомобиля и найдем в интернете соответствующий вид топлива
- приведем данные к единообразию

#### Для значений '0', '1,6', '2'
- проверим строки, насколько соответствует отображение данных по столбцам
- найдем марку и модель автомобиля и найдем в интернете коответствующий объем двигателя

In [ ]:
data.query('вид_топлива == "0"')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
37121,2019,Январь,БИПЭК АВТО,UAZ,3741,3741,2018,Республика Казахстан,0,2.693,5 МТ,Полный,Развозные автомобили,Талдыкорган,БИПЭК АВТО Талдыкорган,NaN,1.0,14200.0,14200.0,Алматинская область,Коммерческие автомобили,Малотоннажные грузовики
37122,2019,Январь,БИПЭК АВТО,UAZ,3741,3741,2018,Республика Казахстан,0,2.693,5 МТ,Полный,Развозные автомобили,Актобе,БИПЭК АВТО Актобе,NaN,3.0,14200.0,42600.0,Актюбинская область,Коммерческие автомобили,Малотоннажные грузовики
37123,2019,Январь,БИПЭК АВТО,UAZ,3741,3741,2018,Республика Казахстан,0,2.693,5 МТ,Полный,Развозные автомобили,Актау,БИПЭК АВТО Актау,NaN,2.0,14200.0,28400.0,Мангистауская область,Коммерческие автомобили,Малотоннажные грузовики


[Сайт](uaz.ru/cars/classic) сообщает нам, что этот автомобиль оснащается (только) бензиновым двигателем.  
Внесем изменения в датафрейм

In [ ]:
data.loc[(data['бренд'] == "UAZ") & (data['модель'] == "3741"), 'вид_топлива'] = 'p'

In [ ]:

data.loc[(data['бренд'] == "UAZ") & (data['модель'] == "3741"), 'вид_топлива'] = \
    data.loc[(data['бренд'] == "UAZ") & (data['модель'] == "3741"), 'вид_топлива'].str.replace('0', 'p')

In [ ]:
data.query('вид_топлива == "1,6"')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
23728,2019,Январь,Renault Россия,Renault,Duster,E1FB4JA,2018,Российская Федерация,"1,6",MT,4WD,4WD,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23733,2019,Январь,Renault Россия,Renault,Duster,E1FB2JA,2018,Российская Федерация,"1,6",MT,4WD,4WD,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23734,2019,Январь,Renault Россия,Renault,Duster,E0DB2JA,2018,Российская Федерация,"1,6",AT,Передний,Передний,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23735,2019,Январь,Renault Россия,Renault,Duster,E0DB2JA,2018,Российская Федерация,"1,6",AT,Передний,Передний,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23736,2019,Январь,Renault Россия,Renault,Duster,E0DB2JA,2018,Российская Федерация,"1,6",AT,Передний,Передний,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23751,2019,Январь,Renault Россия,Renault,Duster,E0DB2JA,2018,Российская Федерация,"1,6",AT,Передний,Передний,0,Усть-Каменогорск,БИПЭК АВТО Усть-Каменогорск,NaN,1.0,12249.61125,12249.61125,Восточно-Казахстанская область,Внедорожники,Субкомпактные SUV
23754,2019,Январь,Renault Россия,Renault,Duster,E1FB4JA,2018,Российская Федерация,"1,6",MT,4WD,4WD,0,Алматы,ASTER AUTO Алматы,NaN,1.0,12249.61125,12249.61125,г.Алматы,Внедорожники,Субкомпактные SUV
23763,2019,Январь,Renault Россия,Renault,Duster,E1FB4JA,2018,Российская Федерация,"1,6",MT,4WD,4WD,0,Нур-Султан,Автоцентр Астэк,NaN,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23764,2019,Январь,Renault Россия,Renault,Duster,E1FB4JA,2018,Российская Федерация,"1,6",MT,4WD,4WD,0,Петропавловск,Автоцентр Астэк,NaN,1.0,12249.61125,12249.61125,Северо-Казахстанская область,Внедорожники,Субкомпактные SUV
24997,2019,Январь,Renault Россия,Renault,Sandero,SXP16PA5RB,2018,Российская Федерация,"1,6",0,0,0,0,Атырау,Урал-Кров Авто,NaN,1.0,12476.89637,12476.89637,Атырауская область,Легковые автомобили,B класс


[Сайт](https://www.drom.ru/catalog/renault/duster/320471/) сообщает нам, что этот автомобиль Renault модели Duster оснащается (только) бензиновым двигателем.  
Внесем изменения в датафрейм

In [ ]:
data.loc[(data['бренд'] == "Renault") & (data['модель'] == "Duster")\
        & (data['вид_топлива'] == "1,6"), 'вид_топлива'] = 'p'

Для автомобиля Renault модели Sandero столбцы 'объём_двиг_л' и 'коробка_передач' "съехали" влево на одну позицию - исправим это. **Забегая вперед внесем исправления во всем датафрейме**  
Данные из столбца 'вид_топлива' потеряны.  
[Сайт](https://www.drom.ru/catalog/renault/sandero/211452/) сообщает нам, что этот автомобиль Renault модели Sandero оснащается (только) бензиновым двигателем.  
Внесем изменения в датафрейм.

In [ ]:
data.loc[(data['объём_двиг_л'] == "MT") | (data['объём_двиг_л'] == "AT"), 'коробка_передач'] = \
    data.loc[(data['объём_двиг_л'] == "MT") | (data['объём_двиг_л'] == "AT"), 'объём_двиг_л']

In [ ]:
data.loc[(data['объём_двиг_л'] == "MT") | (data['объём_двиг_л'] == "AT"), 'объём_двиг_л'] = \
    data.loc[(data['объём_двиг_л'] == "MT") | (data['объём_двиг_л'] == "AT"), 'вид_топлива']

In [ ]:
data.loc[(data['бренд'] == "Renault") & (data['модель'] == "Sandero")\
        & (data['вид_топлива'] == "1,6"), 'вид_топлива'] = 'Бензин'

In [ ]:
data.query('вид_топлива == "2"')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
23723,2019,Январь,Renault Россия,Renault,Duster,E2GB4AG,2019,Российская Федерация,2,2,MT,4WD,0,Уральск,Урал-Кров Авто,NaN,1.0,12249.61125,12249.61125,Западно-Казахстанская область,Внедорожники,Субкомпактные SUV
23724,2019,Январь,Renault Россия,Renault,Duster,E2GB4AGA,2018,Российская Федерация,2,2,AT,4WD,0,Нур-Султан,Кристалл Авто Астана,NaN,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23725,2019,Январь,Renault Россия,Renault,Duster,E2GB4AGA,2018,Российская Федерация,2,2,AT,4WD,0,Нур-Султан,Кристалл Авто Астана,NaN,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23726,2019,Январь,Renault Россия,Renault,Duster,E2GB4AGA,2018,Российская Федерация,2,2,AT,4WD,0,Кызылорда,Кристалл Авто Кызылорда,NaN,1.0,12249.61125,12249.61125,Кызылординская область,Внедорожники,Субкомпактные SUV
23727,2019,Январь,Renault Россия,Renault,Duster,E2GB4AG,2018,Российская Федерация,2,2,MT,4WD,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23729,2019,Январь,Renault Россия,Renault,Duster,E1FB4AGA,2018,Российская Федерация,2,2,AT,4WD,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23730,2019,Январь,Renault Россия,Renault,Duster,E1FB4AGA,2018,Российская Федерация,2,2,AT,4WD,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23731,2019,Январь,Renault Россия,Renault,Duster,E1FB4AG,2018,Российская Федерация,2,2,MT,4WD,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23732,2019,Январь,Renault Россия,Renault,Duster,E1FB4AG,2018,Российская Федерация,2,2,MT,4WD,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23746,2019,Январь,Renault Россия,Renault,Duster,E2PGB4AGA,2018,Российская Федерация,2,2,AT,4WD,0,Усть-Каменогорск,БИПЭК АВТО Усть-Каменогорск,NaN,1.0,12249.61125,12249.61125,Восточно-Казахстанская область,Внедорожники,Субкомпактные SUV


[Сайт](https://www.drom.ru/catalog/renault/duster/2018/#russia) сообщает нам, что этот автомобиль Renault модели Duster  оснащается (только) бензиновым двигателем.  
Внесем изменения в датафрейм

In [ ]:
data.loc[(data['бренд'] == "Renault") & (data['модель'] == "Duster")\
        & (data['вид_топлива'] == "2"), 'вид_топлива'] = 'p'

#### Для значений `nan`
- найдем марку и модель автомобиля и найдем в интернете коответствующий объем двигателя

<div style="border-radius: 45px;border:solid red 7px; padding: 40px">

Сюда применитьь моду
    

In [ ]:
data.loc[(data['вид_топлива'].isna())]#.head()


,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
861,2019,Март,СемАЗ,Foton,BJ3253DMPKB-AD,"Грузовой автомобиль марки ""Foton BJ3253DMPKB-A...",2019,Республика Казахстан,NaN,NaN,NaN,NaN,NaN,Алматы,Меридиан АВТО Алматы,NaN,1.0,48275.86207,48275.86207,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
862,2019,Июнь,СемАЗ,Foton,BJ3253DMPKB-AD,"Грузовой автомобиль марки ""Foton BJ3253DMPKB-A...",2018,Республика Казахстан,NaN,NaN,NaN,NaN,NaN,Усть-Каменогорск,СемАЗ,NaN,1.0,49735.91972,49735.91972,Восточно-Казахстанская область,Коммерческие автомобили,Крупнотоннажные грузовики
863,2019,Июнь,СемАЗ,Foton,BJ3253DMPKB-AD,"Грузовой автомобиль марки ""Foton BJ3253DMPKB-A...",2018,Республика Казахстан,NaN,NaN,NaN,NaN,NaN,Кокшетау,Вираж Кокшетау,NaN,1.0,48784.39304,48784.39304,Акмолинская область,Коммерческие автомобили,Крупнотоннажные грузовики
864,2019,Июль,СемАЗ,Foton,BJ3253DMPKB-AD,"Грузовой автомобиль марки ""Foton BJ3253DMPKB-A...",2018,Республика Казахстан,NaN,NaN,NaN,NaN,NaN,Алматы,Вираж Алматы,NaN,5.0,48725.42295,243627.11480,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
865,2019,Август,СемАЗ,Foton,BJ3253DMPKB-AD,"Грузовой автомобиль марки ""Foton BJ3253DMPKB-A...",2018,Республика Казахстан,NaN,NaN,NaN,NaN,NaN,Алматы,СемАЗ,NaN,2.0,31031.65229,62063.30457,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39943,2019,Сентябрь,Renault Россия,Renault,Arkana,B3P4M3TX5C,2019,Российская Федерация,NaN,NaN,NaN,NaN,NaN,Нур-Султан,Кристалл Авто Астана,NaN,1.0,21327.27747,21327.27747,г.Нур-Султан,Внедорожники,Компактные SUV
39944,2019,Сентябрь,Renault Россия,Renault,Arkana,B32M1TX5C,2019,Российская Федерация,NaN,NaN,NaN,NaN,NaN,Уральск,Урал-Кров Авто,NaN,1.0,20889.17526,20889.17526,Западно-Казахстанская область,Внедорожники,Компактные SUV
39945,2019,Сентябрь,Renault Россия,Renault,Arkana,B32M1TX5C,2019,Российская Федерация,NaN,NaN,NaN,NaN,NaN,Атырау,Урал-Кров Авто,NaN,1.0,20889.17526,20889.17526,Атырауская область,Внедорожники,Компактные SUV
39946,2019,Сентябрь,Renault Россия,Renault,Arkana,B32M1TX5C,2019,Российская Федерация,NaN,NaN,NaN,NaN,NaN,Уральск,Урал-Кров Авто,NaN,1.0,20889.17526,20889.17526,Западно-Казахстанская область,Внедорожники,Компактные SUV


Более 3000 записей не имеют данных о виде топлива.  
Какие марки здесь присутствуют?

In [ ]:
data.loc[(data['вид_топлива'].isna()), 'модификация'].unique()

array(['Грузовой автомобиль марки "Foton BJ3253DMPKB-AD" (25 тонн)',
       'GAZ 23107', 'GAZ 2705', 'GAZ 27055', 'GAZ 27057', 'GAZ 27527',
       'GAZ 3221', 'GAZ 3302', 'GAZ 33025', 'GAZ 33027', 'GAZ 330202',
       'GAZ 330232', 'GAZ 330252', 'GAZ 330253', 'GAZ 330273',
       'GAZ 33086', 'GAZ A63R42', 'GAZ A63R45', 'GAZ A64R45',
       'GAZ A21R22', 'GAZ A21R23', 'GAZ A21R32', 'GAZ A22R32',
       'GAZ A22R35', 'GAZ A32R32', 'GAZ A21R33', 'GAZ A21R35',
       'GAZ A31R23', 'GAZ A31R33', 'GAZ A32R23', 'GAZ A32R33',
       'GAZ A65R35', 'H350 BUS', 'H350 Ambulance', 'H350 VAN 6.2', nan,
       'HD 160', 'HD65 Euro 5', 'HD65 Euro 4 KZ', 'Euro 5', 'Euro 4',
       'HD78 Euro 5', 'HD78/ЕХ8', 'Luxury', 'Universe Luxury', '70C15',
       '50C15V', '45C15V (Автобус 14+1)', '50С15 (изотермический фургон)',
       'HighRoof', '6х4', 'Intelligent', 'Luxury 1.6 MT',
       'Intelligent 1.6 CVT', 'FL Luxury', 'FL Intelligent',
       'Luxury 2.0 MT5', 'Intelligent 2.0T MT6', 'TGS 26.400 6x4 BL

In [ ]:
list_of_models = data.loc[(data['вид_топлива'].isna()), 'бренд'].unique()

In [ ]:
len(list_of_models)

19

In [ ]:
x = 0
for i in list_of_models:
    print('Бренд', i, "Модель", data.loc[(data['вид_топлива'].isna()) & (data['бренд'] == i), 'модель'].unique())

    x = x + len (data.loc[(data['вид_топлива'].isna()) & (data['бренд'] == i), 'модель'].unique())
print('Общее количество моделей без указания вида топлива', x)

Бренд Foton Модель ['BJ3253DMPKB-AD' 'BJ1069VDJEA-F1']
Бренд GAZ Модель ['2310' '2705' '2752' '3221' '3302' '3308' 'Next']
Бренд Hyundai Truck & Bus Модель ['H350' 'HD 160' 'HD 65' 'HD 78' 'Universe' 'HD 35' 'EX8']
Бренд Iveco Модель ['Daily' 'Daily A' 'Daily V' 'Stralis' 'Tipper']
Бренд Jac Модель ['S3' 'S5' 'N120' 'N75' 'T6' 'N56']
Бренд Jaguar Модель ['XJ' 'E-PACE' 'I-Pace']
Бренд Lada Модель ['ВИС']
Бренд Land Rover Модель ['Discovery Sport' 'Range Rover' 'Range Rover Evoque' 'Range Rover Sport'
 'Range Rover Velar']
Бренд MAN Модель ['TG']
Бренд Mazda Модель ['3' '6' 'CX-5' 'CX-9']
Бренд Peugeot Модель ['301' 'Partner Panel Van']
Бренд Renault Модель ['Duster' 'Kaptur' 'Koleos' 'Logan' 'Master' 'Sandero' 'Dokker' 'Arkana']
Бренд UAZ Модель ['3151' 'Cargo' 'Patriot Pickup']
Бренд Volkswagen Модель ['Amarok' 'Caravelle' 'Polo' 'Tiguan' 'Touareg' 'Transporter Kasten'
 'Teramont']
Бренд Volvo Модель ['XC90' 'XC60']
Бренд Урал Модель ['4320' '4320-1951-40' '4320-1951-60' '5557' '32551'

In [ ]:
for i in list_of_models:
    k = data.loc[(data['вид_топлива'].isna()) & (data['бренд'] == i), 'модель']
    m = data.loc[(data['вид_топлива'].isna()) & (data['бренд'] == i), 'объём_двиг_л']
    l = data.loc[(data['вид_топлива'].isna()) & (data['бренд'] == i), 'коробка_передач']
    n = data.loc[(data['вид_топлива'].isna()) & (data['бренд'] == i), 'тип_привода']
    print(k, m, l, n)


861      BJ3253DMPKB-AD
862      BJ3253DMPKB-AD
863      BJ3253DMPKB-AD
864      BJ3253DMPKB-AD
865      BJ3253DMPKB-AD
866      BJ3253DMPKB-AD
867      BJ3253DMPKB-AD
868      BJ3253DMPKB-AD
869      BJ3253DMPKB-AD
39420    BJ1069VDJEA-F1
39421    BJ1069VDJEA-F1
39422    BJ1069VDJEA-F1
39423    BJ1069VDJEA-F1
39424    BJ1069VDJEA-F1
39425    BJ1069VDJEA-F1
39426    BJ1069VDJEA-F1
39427    BJ1069VDJEA-F1
39428    BJ1069VDJEA-F1
39429    BJ1069VDJEA-F1
39430    BJ1069VDJEA-F1
39431    BJ1069VDJEA-F1
39432    BJ1069VDJEA-F1
39433    BJ1069VDJEA-F1
39434    BJ1069VDJEA-F1
39435    BJ1069VDJEA-F1
39436    BJ1069VDJEA-F1
39437    BJ1069VDJEA-F1
39438    BJ1069VDJEA-F1
Name: модель, dtype: object 861      NaN
862      NaN
863      NaN
864      NaN
865      NaN
866      NaN
867      NaN
868      NaN
869      NaN
39420    NaN
39421    NaN
39422    NaN
39423    NaN
39424    NaN
39425    NaN
39426    NaN
39427    NaN
39428    NaN
39429    NaN
39430    NaN
39431    NaN
39432    NaN
39433    NaN
3

<div style="border-radius: 45px;border:solid red 7px; padding: 40px">
    
    Общее количество моделей без указания вида топлива составляет 72 модели.  
    Это довольно трудозатратно, чтобы выяснять в интернете для  каждой из них на вид топлива.  
    ТЗ не ставит задачу исследование рынка в зависимости от вида топлива автомобиля.  
    Отложим пока за ненадобностью заполнение данных в столбце "вид топлива"
    
    так же отметим, что для заполнения на основе имеющихся данных нужна предобработка столбцов `объём_двиг_л`, `коробка_передач`, `тип_привода`, `сегмент`. Заполнение данных при необходимости произведем ПОСЛЕ обработки значений в этих столбцах

#### Приведем данные к единообразию


In [ ]:
# приведем все значения к нижнему регистру и удалим пробелы
data.вид_топлива = data.вид_топлива.str.lower().str.replace(' ', '')

In [ ]:
data['вид_топлива'].unique()

array(['бензин', 'дизель', nan, 'гибрид', 'p', 'электро', 'электричество'],
      dtype=object)

In [ ]:
# Вспомним, что в больших объемах данныз скорость обработки заметно увеличивается
#для более длинных значений по сравнению с короткими.
# примем короткие условные обозначения вида топлива
# Создадим словарь для замены
#dict_type = {'бензин': 'Бензин', 'дизель':'Дизель', 'электричество': 'Электро', 'электро': 'Электро', 'гибрид':'Гибрид'}

In [ ]:
# Вспомним, что в больших объемах данныз скорость обработки заметно увеличивается
#для более длинных значений по сравнению с короткими.
# примем короткие условные обозначения вида топлива
# Создадим словарь для замены
dict_type = {'бензин': 'p', 'дизель':'d', 'электричество': 'e', 'электро': 'e', 'гибрид':'g'}


In [ ]:
# Призведем замену
data['вид_топлива'] = data['вид_топлива'].map(dict_type)

In [ ]:
data['вид_топлива'].unique()

array(['p', 'd', nan, 'g', 'e'], dtype=object)

### Обработка данных в столбце `объём_двиг_л`
Выведем на экран уникальные значения, указанные в столбце `объём_двиг_л`

In [ ]:
sorted(data.объём_двиг_л.unique().astype(str))

['#Н/Д',
 '0',
 '1,6',
 '1.2',
 '1.248',
 '1.2T',
 '1.33',
 '1.4',
 '1.485',
 '1.4Turbo',
 '1.5',
 '1.591',
 '1.596',
 '1.598',
 '1.6',
 '1.69',
 '1.6MPI',
 '1.6T-GDI',
 '1.7',
 '1.774',
 '1.8',
 '1.998',
 '1.999',
 '10.5',
 '10.7',
 '11',
 '11.7',
 '12',
 '12.3',
 '12.7',
 '12.8',
 '13',
 '13.7',
 '14.7',
 '15.7',
 '16.7',
 '17.7',
 '18.7',
 '19.7',
 '2',
 '2.0',
 '2.0CRDI',
 '2.0MPI',
 '2.0h',
 '2.2',
 '2.359',
 '2.4',
 '2.4G',
 '2.4GDI',
 '2.4MPI',
 '2.5',
 '2.5CRDIVGT',
 '2.5CRDIWGT',
 '2.693',
 '2.7',
 '2.8',
 '2.9',
 '20.7',
 '21.7',
 '22.7',
 '23.7',
 '24.7',
 '25.7',
 '26.7',
 '3',
 '3.0L',
 '3.342',
 '3.47',
 '3.5',
 '3.6',
 '3.7',
 '3.8',
 '3.9',
 '4',
 '4.0',
 '4.3',
 '4.3.',
 '4.4',
 '4.6',
 '4.7',
 '4.9',
 '4.98',
 '4.98L.',
 '400Л.С.',
 '5',
 '5.2',
 '5.5',
 '5.6',
 '5.7',
 '6.2',
 '6.5',
 '6.6',
 '6.7',
 '6.7L',
 '7.5',
 '7.6',
 '7.7',
 '8.4L.',
 '8.7',
 '88KWH',
 '9.7',
 'nan',
 'p']

"Почистим" данные:
- '400Л.С.', '88KWH' , 'MT', 'AT', 0, 'Бензин', а также значения более 10 литров
    - проверим строки, насколько соответствует отображение данных по столбцам
    - найдем марку и модель автомобиля и найдем в интернете соответствующий объем двигателя
- nan и '#Н/Д'-   найдем марку и модель автомобиля и найдем в интернете коответствующий объем двигателя
- удалим буквенные символы после цифр


#### Для значений 400Л.С.' и '88KWH'
- проверим строки, насколько соответствует отображение данных по столбцам
- найдем марку и модель автомобиля и найдем в интернете коответствующий объем двигателя

In [ ]:
data.query('объём_двиг_л == "400Л.С."')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
39544,2019,Февраль,Almaty Motors Premium,Jaguar,I-Pace,ЭЛЕКТРОКАР,2019,UK,e,400Л.С.,РЕДУКТОР,4WD,SUV,Алматы,Almaty Motors Premium,Физ. Лицо,1.0,117236.9729,117236.9729,г.Алматы,Внедорожники,Компактные SUV
39545,2019,Апрель,Almaty Motors Premium,Jaguar,I-Pace,ЭЛЕКТРОКАР,2019,UK,e,400Л.С.,РЕДУКТОР,4WD,SUV,Алматы,Almaty Motors Premium,Физ. Лицо,1.0,105388.7924,105388.7924,г.Алматы,Внедорожники,Компактные SUV
39546,2019,Июнь,Almaty Motors Premium,Jaguar,I-Pace,ЭЛЕКТРОКАР,2019,UK,e,400Л.С.,РЕДУКТОР,4WD,SUV,Алматы,Almaty Motors Premium,Юр. Лицо,1.0,120413.2793,120413.2793,г.Алматы,Внедорожники,Компактные SUV


Согласно [Википедии](https://ru.wikipedia.org/wiki/Jaguar_I-Pace) автомобиль Jaguar модели I-Pace является электромобилем. соответственно, не имеет ДВС и не может иметь характеристили `Объем двигателя`, поскольку последняя означает объем двигателя внутреннего сгорания. К ячейке указана мощность электрического двигателя, которым оснащен автомобиль.


In [ ]:
data.query('объём_двиг_л == "88KWH"')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
39818,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,88KWH,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39819,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,88KWH,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39820,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,88KWH,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39821,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,88KWH,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39822,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,88KWH,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39823,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,88KWH,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39824,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,88KWH,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39825,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,88KWH,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39826,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,88KWH,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39827,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,88KWH,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы


Согласно [Сайта yomobile.ru](https://yomobile.ru/elektromobil/hff/ankai-hff6124g03ev.html) указанное транспортное средство является электрическим городским автобусом (электробусом). Соответственно, не имеет ДВС и не может иметь характеристили `Объем двигателя`, поскольку последняя означает объем двигателя внутреннего сгорания. К ячейке указана мощность электрического двигателя, которым оснащен автобус.


Учитывая сказанное:
- приравняем значения в соответствующих ячейках к значению 0.
- Укажем в столбце "вид_топлива" значение "Электро"

In [ ]:
data.loc[(data['объём_двиг_л'] == "400Л.С."), 'вид_топлива'] = 'e'

In [ ]:
data.loc[(data.объём_двиг_л == "88KWH"),'объём_двиг_л'] = 0
data.loc[(data.объём_двиг_л == "400Л.С."),'объём_двиг_л'] = 0

In [ ]:
data['объём_двиг_л'].unique()

array(['1.4', '2', '3', '4.4', '6.6', '1.5', '1.596', '2.4', '1.7',
       '26.7', '25.7', '24.7', '23.7', '22.7', '21.7', '20.7', '19.7',
       '18.7', '17.7', '16.7', '15.7', '14.7', '13.7', '12.7', '11.7',
       '10.7', '9.7', '8.7', '7.7', '6.7', '5.7', '4.7', '3.7', '2.7',
       '6.2', '1.4Turbo', '2.5', '1.6', nan, '2.8', '4.3', '4.3.',
       '1.6MPI', '2.0MPI', '2.4GDI', '2.4MPI', '2.5CRDIVGT', '2.5CRDIWGT',
       '3.9', '7.5', '12.3', '1.6T-GDI', '2.0CRDI', '2.0', '3.5', '5.6',
       '5.2', '3.0L', '5', '1.598', '1.248', '1.998', '2.359', '1.999',
       '3.342', '1.591', '3.47', '1.69', '1.774', '4.6', '1.8', '10.5',
       '4', '5.5', '12', '12.8', '11', '2.2', '1.2', '1.485', 'p', '0',
       '1,6', '13', '1.33', '#Н/Д', '4.0', '2.4G', '2.693', '3.8', '1.2T',
       '3.6', '7.6', '4.9', '2.0h', '2.9', 0, '4.98L.', '4.98', '8.4L.',
       '6.7L', '6.5'], dtype=object)

#### Для значений 'MT' и  'AT'
- проверим строки, насколько соответствует отображение данных по столбцам
- найдем марку и модель автомобиля и найдем в интернете коответствующий объем двигателя

In [ ]:
pd.set_option('max_columns', None)

data.query('объём_двиг_л == "MT" or объём_двиг_л == "AT"').head()

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013


Столбцы 'объём_двиг_л' и 'коробка_передач' "съехали" влево на одну позицию - исправим это.  
Данные из столбца 'вид_топлива' потеряны.

In [ ]:
data.loc[(data['объём_двиг_л'] == "MT") | (data['объём_двиг_л'] == "AT"), 'коробка_передач'] = \
    data.loc[(data['объём_двиг_л'] == "MT") | (data['объём_двиг_л'] == "AT"), 'объём_двиг_л']

In [ ]:
data.loc[(data['объём_двиг_л'] == "MT") | (data['объём_двиг_л'] == "AT"), 'объём_двиг_л'] = \
    data.loc[(data['объём_двиг_л'] == "MT") | (data['объём_двиг_л'] == "AT"), 'вид_топлива']

In [ ]:
sorted(data['объём_двиг_л'].unique().astype(str))

['#Н/Д',
 '0',
 '0',
 '1,6',
 '1.2',
 '1.248',
 '1.2T',
 '1.33',
 '1.4',
 '1.485',
 '1.4Turbo',
 '1.5',
 '1.591',
 '1.596',
 '1.598',
 '1.6',
 '1.69',
 '1.6MPI',
 '1.6T-GDI',
 '1.7',
 '1.774',
 '1.8',
 '1.998',
 '1.999',
 '10.5',
 '10.7',
 '11',
 '11.7',
 '12',
 '12.3',
 '12.7',
 '12.8',
 '13',
 '13.7',
 '14.7',
 '15.7',
 '16.7',
 '17.7',
 '18.7',
 '19.7',
 '2',
 '2.0',
 '2.0CRDI',
 '2.0MPI',
 '2.0h',
 '2.2',
 '2.359',
 '2.4',
 '2.4G',
 '2.4GDI',
 '2.4MPI',
 '2.5',
 '2.5CRDIVGT',
 '2.5CRDIWGT',
 '2.693',
 '2.7',
 '2.8',
 '2.9',
 '20.7',
 '21.7',
 '22.7',
 '23.7',
 '24.7',
 '25.7',
 '26.7',
 '3',
 '3.0L',
 '3.342',
 '3.47',
 '3.5',
 '3.6',
 '3.7',
 '3.8',
 '3.9',
 '4',
 '4.0',
 '4.3',
 '4.3.',
 '4.4',
 '4.6',
 '4.7',
 '4.9',
 '4.98',
 '4.98L.',
 '5',
 '5.2',
 '5.5',
 '5.6',
 '5.7',
 '6.2',
 '6.5',
 '6.6',
 '6.7',
 '6.7L',
 '7.5',
 '7.6',
 '7.7',
 '8.4L.',
 '8.7',
 '9.7',
 'nan',
 'p']

#### Для значений 'p' - Бензин
- проверим строки, насколько соответствует отображение данных по столбцам
- найдем марку и модель автомобиля и найдем в интернете коответствующий объем двигателя

In [ ]:
data.query('объём_двиг_л == "p"').head()

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
23728,2019,Январь,Renault Россия,Renault,Duster,E1FB4JA,2018,Российская Федерация,NaN,p,MT,4WD,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23733,2019,Январь,Renault Россия,Renault,Duster,E1FB2JA,2018,Российская Федерация,NaN,p,MT,4WD,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23734,2019,Январь,Renault Россия,Renault,Duster,E0DB2JA,2018,Российская Федерация,NaN,p,AT,Передний,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23735,2019,Январь,Renault Россия,Renault,Duster,E0DB2JA,2018,Российская Федерация,NaN,p,AT,Передний,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23736,2019,Январь,Renault Россия,Renault,Duster,E0DB2JA,2018,Российская Федерация,NaN,p,AT,Передний,0,Караганда,Кристалл Авто Караганда,NaN,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV


Столбец 'вид_топлива' "съехал" вправо на одну позицию - исправим это.  
Данные из столбца 'объём_двиг_л' потеряны.

In [ ]:
data.loc[(data['объём_двиг_л'] == "p"), 'вид_топлива'] = \
    data.loc[(data['объём_двиг_л'] == "p"), 'объём_двиг_л']

Очистим данные в столбце 'объём_двиг_л'

In [ ]:
data.loc[(data['объём_двиг_л'] == "p"), 'объём_двиг_л'] = ''


#### Для значений '#Н/Д'
- проверим строки, насколько соответствует отображение данных по столбцам
- найдем марку и модель автомобиля и найдем в интернете коответствующий объем двигателя

In [ ]:
data.query('объём_двиг_л == "#Н/Д"')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
31148,2019,Февраль,Toyota Motor Kazakhstan,Toyota,Corolla,7,2019,Турция,p,#Н/Д,#Н/Д,#Н/Д,C класс,Нур-Султан,Тойота Центр Астана,Физ. Лицо,1.0,42316.46043,42316.46043,г.Нур-Султан,Легковые автомобили,C класс
31149,2019,Февраль,Toyota Motor Kazakhstan,Toyota,Corolla,7,2019,Турция,p,#Н/Д,#Н/Д,#Н/Д,C класс,Нур-Султан,Тойота Центр Астана,Физ. Лицо,1.0,42316.46043,42316.46043,г.Нур-Султан,Легковые автомобили,C класс
32670,2019,Апрель,Toyota Motor Kazakhstan,Toyota,HILUX DC,JT,2019,Таиланд,p,#Н/Д,#Н/Д,#Н/Д,Пикапы,Алматы,Toyota City,Юр. Лицо,1.0,30870.28194,30870.28194,г.Алматы,Пикапы,Pick-ups
33070,2019,Июль,Toyota Motor Kazakhstan,Toyota,HILUX DC,NX,2019,Таиланд,p,#Н/Д,#Н/Д,#Н/Д,Пикапы,Атырау,Тойота Центр Атырау,Физ. Лицо,1.0,27012.45730,27012.45730,Атырауская область,Пикапы,Pick-ups


Данные отсутствуют. Мы удалим их чуть позже, при обработке нечисловых значений в столбце

#### Для значений '0'
- проверим строки, насколько соответствует отображение данных по столбцам
- найдем марку и модель автомобиля и найдем в интернете коответствующий объем двигателя

In [ ]:
data.query('объём_двиг_л == "0"')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
24997,2019,Январь,Renault Россия,Renault,Sandero,SXP16PA5RB,2018,Российская Федерация,p,0,0,0,0,Атырау,Урал-Кров Авто,NaN,1.0,12476.89637,12476.89637,Атырауская область,Легковые автомобили,B класс
25002,2019,Январь,Renault Россия,Renault,Sandero,SUT16PA5RB,2018,Российская Федерация,p,0,0,0,0,Атырау,Урал-Кров Авто,NaN,1.0,12476.89637,12476.89637,Атырауская область,Легковые автомобили,B класс
25023,2019,Январь,Renault Россия,Renault,Sandero,SXP16PA5RB,2018,Российская Федерация,p,0,0,0,0,Нур-Султан,Автоцентр Астэк,NaN,1.0,12476.89637,12476.89637,г.Нур-Султан,Легковые автомобили,B класс


Согласно [Сайта](https://www.drom.ru/catalog/renault/sandero/specs/transmission/) Рено Sandero 2018 г.в. комплектовался несколькими видами бензиновыз двигателей с одинаковым объемом - 1.6 литров и передним приводом. КПП могла быть автоматической и механической.   


In [ ]:
data.loc[(data['модель'] == 'Sandero'), 'вид_топлива'] = "p"
data.loc[(data['модель'] == 'Sandero'), 'объём_двиг_л'] = 1.6
data.loc[(data['модель'] == 'Sandero'), 'тип_привода'] = "FWD"

In [ ]:
#data.loc[(data['объём_двиг_л'] == "0"), 'объём_двиг_л'] = ''

#### Удалим буквенные символы после цифр
~Результат запишем в новый столбец `объём_двиг_л_next`~

In [ ]:
for i in tqdm(range(data.shape[0])):
    try:
        data.loc[(data.index == i), 'объём_двиг_л'] = re.sub("[^0-9.]", "", data.loc[(data.index == i), 'объём_двиг_л'][i])
    except:
        pass

100%|██████████| 39966/39966 [00:48<00:00, 831.54it/s] 


In [ ]:
for i in tqdm(range(data.shape[0])):
    try:
        data.loc[(data.index == i), 'объём_двиг_л'] = re.sub("\.{2,}", "", data.loc[(data.index == i), 'объём_двиг_л'][i])
    except:
        pass

100%|██████████| 39966/39966 [00:46<00:00, 861.82it/s] 


In [ ]:
data['объём_двиг_л'].unique()

array(['1.4', '2', '3', '4.4', '6.6', '1.5', '1.596', '2.4', '1.7',
       '26.7', '25.7', '24.7', '23.7', '22.7', '21.7', '20.7', '19.7',
       '18.7', '17.7', '16.7', '15.7', '14.7', '13.7', '12.7', '11.7',
       '10.7', '9.7', '8.7', '7.7', '6.7', '5.7', '4.7', '3.7', '2.7',
       '6.2', '2.5', '1.6', nan, '2.8', '4.3', '4.3.', '2.0', '3.9',
       '7.5', '12.3', '3.5', '5.6', '5.2', '3.0', '5', '1.598', '1.248',
       '1.998', '2.359', '1.999', '3.342', '1.591', '3.47', '1.69',
       '1.774', '4.6', '1.8', '10.5', '4', '5.5', '12', '12.8', '11',
       '2.2', '1.2', '1.485', '', 1.6, '13', '1.33', '4.0', '2.693',
       '3.8', '3.6', '7.6', '4.9', '2.9', 0, '4.98.', '4.98', '8.4.',
       '6.5'], dtype=object)

In [ ]:
data.query('объём_двиг_л == 0')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
39544,2019,Февраль,Almaty Motors Premium,Jaguar,I-Pace,ЭЛЕКТРОКАР,2019,UK,e,0,РЕДУКТОР,4WD,SUV,Алматы,Almaty Motors Premium,Физ. Лицо,1.0,117236.9729,117236.9729,г.Алматы,Внедорожники,Компактные SUV
39545,2019,Апрель,Almaty Motors Premium,Jaguar,I-Pace,ЭЛЕКТРОКАР,2019,UK,e,0,РЕДУКТОР,4WD,SUV,Алматы,Almaty Motors Premium,Физ. Лицо,1.0,105388.7924,105388.7924,г.Алматы,Внедорожники,Компактные SUV
39546,2019,Июнь,Almaty Motors Premium,Jaguar,I-Pace,ЭЛЕКТРОКАР,2019,UK,e,0,РЕДУКТОР,4WD,SUV,Алматы,Almaty Motors Premium,Юр. Лицо,1.0,120413.2793,120413.2793,г.Алматы,Внедорожники,Компактные SUV
39818,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,0,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39819,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,0,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39820,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,0,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39821,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,0,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39822,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,0,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39823,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,0,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39824,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,Bus,2019,Республика Казахстан,e,0,NaN,2WD,автобус,Костанай,Allur Auto Almaty,Юр. Лицо,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы


Это электромобиль и электроавтобус

<div style="border-radius: 45px;border:solid red 7px; padding: 40px">

Сюда нужно вернуться - в нижеследующий код
    
    засада в виде топлива до этой строки и после 2.6.3
    

In [ ]:
display(round(data.isna().mean() * 100).astype(int))

год                                0
месяц                              0
компания                           0
бренд                              0
модель                             0
модификация                        9
год_выпуска                        1
страна_производитель               0
вид_топлива                        7
объём_двиг_л                      10
коробка_передач                    8
тип_привода                       10
сегмент                           17
регион                             0
наименование_дилерского_центра     0
тип_клиента                       18
количество                         0
цена_usd                           0
продажа_usd                        0
область                            0
сегментация_2013                   0
класс_2013                         0
dtype: int64

In [ ]:
#print(data.loc[(data.объём_двиг_л == "4.9"),'объём_двиг_л'].astype(str).str.split('').str[0])

значения с двумя точками - избавимся от второй точки

In [ ]:
data.loc[(data.объём_двиг_л == "4.3."),'объём_двиг_л'] = 4.3
data.loc[(data.объём_двиг_л == "4.98."),'объём_двиг_л'] = 4.98
data.loc[(data.объём_двиг_л == "8.4."),'объём_двиг_л'] = 8.4

In [ ]:
#data['объём_двиг_л_next'].unique()

In [ ]:
data['объём_двиг_л'].unique()

array(['1.4', '2', '3', '4.4', '6.6', '1.5', '1.596', '2.4', '1.7',
       '26.7', '25.7', '24.7', '23.7', '22.7', '21.7', '20.7', '19.7',
       '18.7', '17.7', '16.7', '15.7', '14.7', '13.7', '12.7', '11.7',
       '10.7', '9.7', '8.7', '7.7', '6.7', '5.7', '4.7', '3.7', '2.7',
       '6.2', '2.5', '1.6', nan, '2.8', '4.3', 4.3, '2.0', '3.9', '7.5',
       '12.3', '3.5', '5.6', '5.2', '3.0', '5', '1.598', '1.248', '1.998',
       '2.359', '1.999', '3.342', '1.591', '3.47', '1.69', '1.774', '4.6',
       '1.8', '10.5', '4', '5.5', '12', '12.8', '11', '2.2', '1.2',
       '1.485', '', 1.6, '13', '1.33', '4.0', '2.693', '3.8', '3.6',
       '7.6', '4.9', '2.9', 0, 4.98, '4.98', 8.4, '6.5'], dtype=object)

##### Приведем к числовому формату столбец   `объём_двиг_л`

In [ ]:
# зададим список столбцов
col_list_capacity = ['объём_двиг_л']

In [ ]:
int_cols = ['объём_двиг_л']
for col in int_cols:
    # downcast='signed' приводит данные к минимально возможному типу числовых данных
    data[col] = pd.to_numeric(data[col], downcast='signed')
print(data[int_cols].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39966 entries, 0 to 39965
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   объём_двиг_л  36130 non-null  float64
dtypes: float64(1)
memory usage: 312.4 KB
None


In [ ]:
data['объём_двиг_л'].unique()

array([ 1.4  ,  2.   ,  3.   ,  4.4  ,  6.6  ,  1.5  ,  1.596,  2.4  ,
        1.7  , 26.7  , 25.7  , 24.7  , 23.7  , 22.7  , 21.7  , 20.7  ,
       19.7  , 18.7  , 17.7  , 16.7  , 15.7  , 14.7  , 13.7  , 12.7  ,
       11.7  , 10.7  ,  9.7  ,  8.7  ,  7.7  ,  6.7  ,  5.7  ,  4.7  ,
        3.7  ,  2.7  ,  6.2  ,  2.5  ,  1.6  ,    nan,  2.8  ,  4.3  ,
        3.9  ,  7.5  , 12.3  ,  3.5  ,  5.6  ,  5.2  ,  5.   ,  1.598,
        1.248,  1.998,  2.359,  1.999,  3.342,  1.591,  3.47 ,  1.69 ,
        1.774,  4.6  ,  1.8  , 10.5  ,  4.   ,  5.5  , 12.   , 12.8  ,
       11.   ,  2.2  ,  1.2  ,  1.485, 13.   ,  1.33 ,  2.693,  3.8  ,
        3.6  ,  7.6  ,  4.9  ,  2.9  ,  0.   ,  4.98 ,  8.4  ,  6.5  ])

#### Для значений более 10 литров
- найдем марку и модель автомобиля и оценим cоответствие объема двигателя

In [ ]:
for i in data.query('объём_двиг_л >= 10')['модель'].unique():
    print(data.query('объём_двиг_л >= 10 and модель == @i')[['бренд', 'модель', 'объём_двиг_л', 'сегмент']].head(2))

         бренд модель  объём_двиг_л        сегмент
399  Chevrolet   Niva          26.7  Внедорожники 
400  Chevrolet   Niva          25.7  Внедорожники 
                    бренд    модель  объём_двиг_л        сегмент
9699  Hyundai Truck & Bus  Universe          12.3  туристический
9700  Hyundai Truck & Bus  Universe          12.3  туристический
      бренд модель  объём_двиг_л сегмент
20212   MAN     TG          10.5     NaN
20213   MAN     TG          10.5     NaN
                      бренд  модель  объём_двиг_л                    сегмент
20386  Mercedes-Benz Trucks  Actros          12.0    Коммерческие автомобили
20387  Mercedes-Benz Trucks  Actros          12.8  Крупнотоннажные грузовики
                      бренд    модель  объём_двиг_л           сегмент
20402  Mercedes-Benz Trucks  Tourismo          12.0  Большие автобусы
20403  Mercedes-Benz Trucks  Tourismo          12.0  Большие автобусы
        бренд    модель  объём_двиг_л                  сегмент
25430  Scania  G-Series  

Итак, у нас но моделей автомобилей различных брендов с объемом двигателя более 10 литров.  
- Из них две модели - это Chevrolet Niva и Renault Sandero представляют собой легковой автомобиль. Будет весьма вероятным, что настолько большой объем двигателя, приписанный им, возник вследствие ошибки - десятичная точка поставлена неправильно. Исправим ситуацию.
- Остальные автомобили в выборке - грузовики, траки, автобусы и вполне могут иметь двигатель с объемом, превышающим 10 литров.

In [ ]:
list_for_division = ['Niva', 'Sandero']

In [ ]:
for i in list_for_division:
    data.loc[((data['объём_двиг_л'] >= 10) & (data['модель'] == i)), 'объём_двиг_л']  = \
        data.loc[(data['объём_двиг_л'] >= 10) & (data['модель'] == i), 'объём_двиг_л'] / 10

   # data.query('объём_двиг_л >= 10 and модель == @i')['объём_двиг_л'] = \
    #    data.query('объём_двиг_л >= 10 and модель == @i')['объём_двиг_л'] / 10
    print(data.query('модель == @i')[['бренд', 'модель', 'год_выпуска', 'объём_двиг_л', 'сегмент']].head(2))

         бренд модель год_выпуска  объём_двиг_л        сегмент
264  Chevrolet   Niva        2017           1.7  Внедорожники 
265  Chevrolet   Niva        2018           1.7  Внедорожники 
         бренд   модель год_выпуска  объём_двиг_л сегмент
24989  Renault  Sandero        2018           1.6       0
24990  Renault  Sandero        2018           1.6       0


Проверим, тем не менее - какие для указанных автомобилей еще занедены данные об объеме двигателя.

Проверим для Sandero

In [ ]:
data.loc[((data['модель'] == "Sandero")), 'объём_двиг_л'].unique()

array([1.6])

Здесь все нормально - указан один литраж - это 1,6 литров

Проверим для Niva

In [ ]:
data.loc[( (data['модель'] == "Niva")), 'объём_двиг_л'].unique()

array([1.7 , 2.67, 2.57, 2.47, 2.37, 2.27, 2.17, 2.07, 1.97, 1.87, 1.77,
       1.67, 1.57, 1.47, 1.37, 1.27, 1.17, 1.07, 9.7 , 8.7 , 7.7 , 6.7 ,
       5.7 , 4.7 , 3.7 , 2.7 ])

Здерь разброс огромный.   
Уточним имеющиеся в базе года выпуска автомобилей и заглянем в интернет

In [ ]:
data.loc[( (data['модель'] == "Niva")), 'год_выпуска'].unique()

array(['2017', '2018', '2019'], dtype=object)

[Сайт auto.ru](https://auto.ru/catalog/cars/chevrolet/niva/specifications/harakteristiki-dvigatelya/) подсказывает нам , что на автомобили Niva с 2002 по 2020 годы устанавливался единственно возможный бензиновый джвигатель объемом 1,69 литров.  
Не откладывая внесем изменения.

In [ ]:
data.loc[(data['модель'] == "Niva") & (data['бренд'] == "Chevrolet"), 'объём_двиг_л'] = 1.69

In [ ]:
data.loc[( (data['модель'] == "Niva")), 'объём_двиг_л'].unique()

array([1.69])

Округлим до одного знака после запятой значения в столбце `объём_двиг_л`

In [ ]:
data['объём_двиг_л'] = round(data['объём_двиг_л'],1)

In [ ]:
data['объём_двиг_л'].unique()

array([ 1.4,  2. ,  3. ,  4.4,  6.6,  1.5,  1.6,  2.4,  1.7,  6.2,  2.5,
        nan,  2.7,  2.8,  4.3,  3.9,  7.5, 12.3,  3.5,  5.6,  8.7,  5.2,
        5. ,  1.2,  3.3,  1.8,  4.6,  5.7, 10.5,  4. ,  5.5, 12. , 12.8,
       11. ,  2.2, 13. ,  1.3,  3.8,  3.6,  7.6,  4.9,  2.9,  0. ,  8.4,
        6.7,  6.5])

#### Для значений `nan`
- найдем марку и модель автомобиля и найдем в интернете cоответствующий объем двигателя

In [ ]:
data.loc[(data['объём_двиг_л'].isna()), 'бренд'].unique()

array(['Foton', 'GAZ', 'Hyundai Truck & Bus', 'Isuzu', 'Iveco', 'Jac',
       'Jaguar', 'KAMAZ', 'Land Rover', 'MAN', 'Mazda',
       'Mercedes-Benz Trucks', 'Peugeot', 'Ravon', 'Renault', 'Toyota',
       'Volkswagen', 'Volvo', 'Урал', 'Shacman', 'Dong Feng', 'ANKAI',
       'Nefaz'], dtype=object)

In [ ]:
data.loc[(data['объём_двиг_л'].isna()), 'модель'].unique()

array(['BJ3253DMPKB-AD', '2310', '2705', '2752', '3221', '3302', '3308',
       'Next', 'H350', 'HD 160', 'HD 65', 'HD 78', 'Universe', 'D-MAX',
       'NMR', 'NPR', 'Daily', 'Daily A', 'Daily V', 'Stralis', 'Tipper',
       'S3', 'S5', 'XJ', '4308', '4311', '43118', '43253', '43502',
       '44108', '45142', '45143', '53215', '53504', '53605', '54115',
       '5490', '65111', '65115', '65116', '65117', '6520', '65206',
       '65225', '66052', '6606', 'Discovery Sport', 'Range Rover',
       'Range Rover Evoque', 'Range Rover Sport', 'TG', '3', '6', 'CX-5',
       'Actros', '301', 'Partner Panel Van', 'Nexia R3', 'R4', 'Duster',
       'Kaptur', 'Koleos', 'Logan', 'Master', 'Corolla', 'HILUX DC',
       'Amarok', 'Caravelle', 'Polo', 'Tiguan', 'Touareg',
       'Transporter Kasten', 'XC90', '4320', '4320-1951-40',
       '4320-1951-60', '5557', 'CX-9', 'HD 35', 'Range Rover Velar',
       'X200', 'E-PACE', 'Dokker', 'N120', 'N75', 'T6', 'SX3258DR384',
       'iev', 'NQR', 'XC60', 'N56

In [ ]:
data.loc[(data['объём_двиг_л'].isna()), 'модификация'].unique()

array(['Грузовой автомобиль марки "Foton BJ3253DMPKB-AD" (25 тонн)',
       'GAZ 23107', 'GAZ 2705', 'GAZ 27055', 'GAZ 27057', 'GAZ 27527',
       'GAZ 3221', 'GAZ 3302', 'GAZ 33025', 'GAZ 33027', 'GAZ 330202',
       'GAZ 330232', 'GAZ 330252', 'GAZ 330253', 'GAZ 330273',
       'GAZ 33086', 'GAZ A63R42', 'GAZ A63R45', 'GAZ A64R45',
       'GAZ A21R22', 'GAZ A21R23', 'GAZ A21R32', 'GAZ A22R32',
       'GAZ A22R35', 'GAZ A32R32', 'GAZ A21R33', 'GAZ A21R35',
       'GAZ A31R23', 'GAZ A31R33', 'GAZ A32R23', 'GAZ A32R33',
       'GAZ A65R35', 'H350 BUS', 'H350 VAN 6.2', 'H350 Ambulance', nan,
       'HD 160', 'HD65 Euro 4 KZ', 'HD65 Euro 5', 'Euro 5', 'Euro 4',
       'HD78 Euro 5', 'HD78/ЕХ8', 'Luxury', 'Universe Luxury', 'D-MAX',
       'Isuzu NMR85', 'Isuzu NPR75', '70C15', '50C15V',
       '45C15V (Автобус 14+1)', '50С15 (изотермический фургон)',
       'HighRoof', '6х4', 'Luxury 1.6 MT', 'Intelligent 1.6 CVT',
       'Intelligent', 'FL Luxury', 'FL Intelligent', 'Luxury 2.0 MT5',
   

In [ ]:
data.loc[(data['объём_двиг_л'].isna()), 'вид_топлива'].unique()

array([nan, 'd', 'p'], dtype=object)

In [ ]:
len(data.loc[(data['объём_двиг_л'].isna()), 'модель'].unique())

106

<div style="border-radius: 45px;border:solid red 7px; padding: 40px">
Для 107 моделей автомобилей не внесен объем двигателя.
    
Это очень трудоемко - заполнять данные из интернета для стольких значений
    
Отложим заполнение этих значений


### Обработка данных в столбце `коробка_передач`
Выведем на экран уникальные значения, указанные в столбце `коробка_передач`

In [ ]:
sorted(data.коробка_передач.unique().astype(str))

[' 7 АКП',
 '#Н/Д',
 '0',
 '12AT',
 '12АТ',
 '4 АТ',
 '4AT',
 '4АТ',
 '5 AT',
 '5 МТ',
 '5AT',
 '5M',
 '5MT',
 '5АТ',
 '5М',
 '5МТ',
 '6 AT',
 '6 DSG',
 '6 MT',
 '6 АТ',
 '6 МТ',
 '6A',
 '6AT',
 '6DSG',
 '6M/T',
 '6MT',
 '6А',
 '6АТ',
 '6МТ',
 '7 DCT',
 '7 DSG',
 '7 АКПП (PDK)',
 '7 АКПП PDK',
 '7AT',
 '7DCT',
 '7DSG',
 '7G-TRONIC',
 '7АКП',
 '7АТ',
 '8',
 '8 AT',
 '8 АКПП (PDK)',
 '8 АКПП Tiptronic S',
 '8 АТ',
 '8-ступ АКПП',
 '8A',
 '8AT',
 '8АКПП',
 '8АТ',
 '9AT',
 '9G-TRONIC',
 'A/T',
 'AMT',
 'AT',
 'CVT',
 'CVT (вариATор)',
 'CVT (вариатор)',
 'CVT(вAриATор)',
 'DCT',
 'DSG',
 'M/T',
 'MT',
 'PDK',
 'Powershift S6',
 'S-Tronic',
 'S-tronic',
 'Steptronic',
 'TDI',
 'Tiptronic',
 'nan',
 'А/T',
 'АT',
 'АКП',
 'АКПП',
 'АТ',
 'М/T',
 'МT',
 'МКП',
 'МКПП',
 'МТ',
 'Мех.',
 'РЕДУКТОР']

"Почистим" данные:

- обработаем исключения - их проверим вручную: '8', '0', 'TDI', 'РЕДУКТОР'
    - проверим строки, насколько соответствует отображение данных по столбцам
    - найдем марку и модель автомобиля и найдем в интернете соответствующий объем двигателя
- обработаем исключение - 'AMT'    
- приведем все значения к нижнему регистру
- всё, что содержит букву `м` на кириллице или `m` на латинице - что соответствует механической коробке передач - обозначим как `МКПП`
- всё остальное будет соответствовать автоматической или автоматизированной, роботизированной коробке передач - обозначим как `АКПП`
- nan и '#Н/Д'-   найдем марку и модель автомобиля и найдем в интернете соответствующий объем двигателя



#### Oбработаем значение '8'

In [ ]:
data.query('коробка_передач == "8"')['модель'].unique()

array(['Cayenne'], dtype=object)

Без дополнительной проверки примем, что у Cayenne автоматическая коробка передач

#### Oбработаем значение '0'

In [ ]:
data.query('коробка_передач == "0"')['модель'].unique()

array(['Sandero'], dtype=object)

In [ ]:
data.query('коробка_передач == "0"')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
24997,2019,Январь,Renault Россия,Renault,Sandero,SXP16PA5RB,2018,Российская Федерация,p,1.6,0,FWD,0,Атырау,Урал-Кров Авто,NaN,1.0,12476.89637,12476.89637,Атырауская область,Легковые автомобили,B класс
25002,2019,Январь,Renault Россия,Renault,Sandero,SUT16PA5RB,2018,Российская Федерация,p,1.6,0,FWD,0,Атырау,Урал-Кров Авто,NaN,1.0,12476.89637,12476.89637,Атырауская область,Легковые автомобили,B класс
25023,2019,Январь,Renault Россия,Renault,Sandero,SXP16PA5RB,2018,Российская Федерация,p,1.6,0,FWD,0,Нур-Султан,Автоцентр Астэк,NaN,1.0,12476.89637,12476.89637,г.Нур-Султан,Легковые автомобили,B класс


Согласно [Сайта](https://www.drom.ru/catalog/renault/sandero/specs/transmission/) Рено Sandero комплектовался и механической и автоматической коробкой передач. У нас отсутствуют какие-либо данные для дополнительной идентификации.   

Какие значения мы имеем для этого автомобиля сейчас?

In [ ]:
data.query('модель == "Sandero"').groupby('коробка_передач').agg({'коробка_передач':'count'})

,коробка_передач
коробка_передач,
0,3
MT,3


Всего 6 значений, половина из которых - Механическая каробка, а вторая половина - не определена.

А сколько всего таких автомобилей в датафрейме?

In [ ]:
len(data.query('модель == "Sandero"'))

441

441 автомобиль.  
Какие значения принимает показатель коробки передач?

In [ ]:
data.query('модель == "Sandero"').коробка_передач.unique()

array([nan, '0', 'MT'], dtype=object)

Получается, что все остальные - то есть основная масса значений - не имеет указания на тип КПП.  
Учитывая данные [Сайта](https://www.drom.ru/catalog/renault/sandero/specs/transmission/) , из которых виидно, что преимущественно автомобиль комплектуется механической коробкой, примем для этого автомобиля и заполним данные о типе коробки передач значением МКПП

In [ ]:
data.loc[(data.модель == "Sandero"),'коробка_передач'] = "МКПП"

#### Oбработаем значение 'TDI'

In [ ]:
data.query('коробка_передач == "TDI"')['бренд'].unique()

array(['Jac'], dtype=object)

In [ ]:
data.query('коробка_передач == "TDI"')['модель'].unique()

array(['T6'], dtype=object)

In [ ]:
#data.query('коробка_передач == "TDI"')['модель'].unique()
data.query('коробка_передач == "TDI"').head()

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
39077,2019,Апрель,Allur Auto,Jac,T6,Comfort,2019,Республика Казахстан,d,2.0,TDI,4WD,PICKUP,Атырау,Рус-Моторс,Юр. Лицо,1.0,21791.24061,21791.24061,Атырауская область,Внедорожники,Компактные SUV
39078,2019,Апрель,Allur Auto,Jac,T6,Comfort,2019,Республика Казахстан,d,2.0,TDI,4WD,PICKUP,Алматы,Allur Auto Almaty,Юр. Лицо,1.0,23686.13110,23686.13110,г.Алматы,Внедорожники,Компактные SUV
39079,2019,Апрель,Allur Auto,Jac,T6,Comfort,2019,Республика Казахстан,d,2.0,TDI,4WD,PICKUP,Алматы,Allur Auto Almaty,Юр. Лицо,1.0,23686.13110,23686.13110,г.Алматы,Внедорожники,Компактные SUV
39080,2019,Апрель,Allur Auto,Jac,T6,Comfort,2019,Республика Казахстан,d,2.0,TDI,4WD,PICKUP,Алматы,Allur Auto Almaty,Юр. Лицо,1.0,23686.13110,23686.13110,г.Алматы,Внедорожники,Компактные SUV
39081,2019,Апрель,Allur Auto,Jac,T6,Comfort,2019,Республика Казахстан,d,2.0,TDI,4WD,PICKUP,Алматы,Allur Auto Almaty,Юр. Лицо,1.0,23686.13110,23686.13110,г.Алматы,Внедорожники,Компактные SUV


Согласно [Официального сайта](https://jaccar.ru/auto/t6.html) Автомобиль комплектуется как бензиновым, так и дизельным двигателями. Оба вида двигателей имеют равный объем - 2 литра.  
В характеристиках производитель указывает единственно возможный тип коробки передач - механическую

Согласно [Википедии](https://en.wikipedia.org/wiki/TDI_(engine)) TDI (непосредственный впрыск с турбонаддувом) - это термин, которым Volkswagen Group обозначают текущую линейку турбодизельных непосредственным впрыском Common Rail, оснащенных промежуточным охладителем в дополнение к турбокомпрессору.  
То есть мы вправе предположить, что при заполнении таблицы по этому автомобилю сотрудник аббревиатурой TDI обозначил тип двигателя (вид топлива) - и это дизельный двигатель.

Заполним значения и коробки передач и вида топлива


In [ ]:
data.loc[(data['коробка_передач'] == "TDI"), 'вид_топлива'] = "d"
data.loc[(data['коробка_передач'] == "TDI"), 'коробка_передач'] = "МКПП"

#### Oбработаем значение 'РЕДУКТОР'

In [ ]:
data.query('коробка_передач == "РЕДУКТОР"')['модель'].unique()
#data.query('коробка_передач == "редуктор"')['модель'].unique()

array(['I-Pace'], dtype=object)

In [ ]:
data.query('коробка_передач == "РЕДУКТОР"')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
39544,2019,Февраль,Almaty Motors Premium,Jaguar,I-Pace,ЭЛЕКТРОКАР,2019,UK,e,0.0,РЕДУКТОР,4WD,SUV,Алматы,Almaty Motors Premium,Физ. Лицо,1.0,117236.9729,117236.9729,г.Алматы,Внедорожники,Компактные SUV
39545,2019,Апрель,Almaty Motors Premium,Jaguar,I-Pace,ЭЛЕКТРОКАР,2019,UK,e,0.0,РЕДУКТОР,4WD,SUV,Алматы,Almaty Motors Premium,Физ. Лицо,1.0,105388.7924,105388.7924,г.Алматы,Внедорожники,Компактные SUV
39546,2019,Июнь,Almaty Motors Premium,Jaguar,I-Pace,ЭЛЕКТРОКАР,2019,UK,e,0.0,РЕДУКТОР,4WD,SUV,Алматы,Almaty Motors Premium,Юр. Лицо,1.0,120413.2793,120413.2793,г.Алматы,Внедорожники,Компактные SUV


Примем, что у электрического Jaguar автоматическая коробка передач

#### Oбработаем значение 'AMT'

In [ ]:
data.loc[(data['коробка_передач'] == 'AMT')]

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
17031,2019,Январь,БИПЭК АВТО,Lada,Granta,Granta,2018,Республика Казахстан,p,1.6,AMT,Передний,B класс,Алматы,БИПЭК АВТО Алматы,NaN,50.0,8700.0,435000.0,г.Алматы,Легковые автомобили,B класс
17102,2019,Февраль,БИПЭК АВТО,Lada,Granta,NaN,2018,Республика Казахстан,p,1.6,AMT,Передний,B класс,Алматы,БИПЭК АВТО Алматы,NaN,50.0,7100.0,355000.0,г.Алматы,Легковые автомобили,B класс
17112,2019,Февраль,БИПЭК АВТО,Lada,Granta,NaN,2019,Республика Казахстан,p,1.6,AMT,Передний,B класс,Актау,БИПЭК АВТО Актау,NaN,1.0,7100.0,7100.0,Мангистауская область,Легковые автомобили,B класс
17213,2019,Апрель,БИПЭК АВТО,Lada,Granta,NaN,2018,Республика Казахстан,p,1.6,AMT,Передний,B класс,Усть-Каменогорск,БИПЭК АВТО Усть-Каменогорск,NaN,1.0,7100.0,7100.0,Восточно-Казахстанская область,Легковые автомобили,B класс
17228,2019,Май,БИПЭК АВТО,Lada,Granta,NaN,2019,Республика Казахстан,p,1.6,AMT,Передний,B класс,Уральск,БИПЭК АВТО Уральск,NaN,1.0,7100.0,7100.0,Западно-Казахстанская область,Легковые автомобили,B класс
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18673,2019,Август,БИПЭК АВТО,Lada,XRAY,NaN,2018,Республика Казахстан,p,1.8,AMT,Передний,B класс,Алматы,БИПЭК АВТО Алматы,NaN,2.0,13800.0,27600.0,г.Алматы,Легковые автомобили,B класс
18695,2019,Сентябрь,БИПЭК АВТО,Lada,XRAY,NaN,2019,Республика Казахстан,p,1.8,AMT,Передний,B класс,Актобе,БИПЭК АВТО Актобе,NaN,1.0,13800.0,13800.0,Актюбинская область,Легковые автомобили,B класс
18696,2019,Сентябрь,БИПЭК АВТО,Lada,XRAY,NaN,2019,Республика Казахстан,p,1.8,AMT,Передний,B класс,Атырау,БИПЭК АВТО Атырау,NaN,2.0,13800.0,27600.0,Атырауская область,Легковые автомобили,B класс
18697,2019,Сентябрь,БИПЭК АВТО,Lada,XRAY,NaN,2019,Республика Казахстан,p,1.8,AMT,Передний,B класс,Петропавловск,БИПЭК АВТО Петропавловск,NaN,3.0,13800.0,41400.0,Северо-Казахстанская область,Легковые автомобили,B класс


In [ ]:
data.loc[(data['коробка_передач'] == "AMT"), 'коробка_передач'] = "АКПП"

#### Oбработаем значение '#Н/Д'

In [ ]:
data.loc[(data['коробка_передач'] == '#Н/Д')]
#data.loc[(data['коробка_передач'] == '#н/д')]

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
31148,2019,Февраль,Toyota Motor Kazakhstan,Toyota,Corolla,7,2019,Турция,p,NaN,#Н/Д,#Н/Д,C класс,Нур-Султан,Тойота Центр Астана,Физ. Лицо,1.0,42316.46043,42316.46043,г.Нур-Султан,Легковые автомобили,C класс
31149,2019,Февраль,Toyota Motor Kazakhstan,Toyota,Corolla,7,2019,Турция,p,NaN,#Н/Д,#Н/Д,C класс,Нур-Султан,Тойота Центр Астана,Физ. Лицо,1.0,42316.46043,42316.46043,г.Нур-Султан,Легковые автомобили,C класс
32670,2019,Апрель,Toyota Motor Kazakhstan,Toyota,HILUX DC,JT,2019,Таиланд,p,NaN,#Н/Д,#Н/Д,Пикапы,Алматы,Toyota City,Юр. Лицо,1.0,30870.28194,30870.28194,г.Алматы,Пикапы,Pick-ups
33070,2019,Июль,Toyota Motor Kazakhstan,Toyota,HILUX DC,NX,2019,Таиланд,p,NaN,#Н/Д,#Н/Д,Пикапы,Атырау,Тойота Центр Атырау,Физ. Лицо,1.0,27012.45730,27012.45730,Атырауская область,Пикапы,Pick-ups


[Сайт DROM](https://www.drom.ru/catalog/toyota/corolla/g_2018_9402/) сообщает нам, что автомобиль Toyota Corolla оснащался двумя видами бензиновых двигателей, имел МКПП и вариатор и выпускался только на переднем приводе.  
Внесем изменения в датасет.

In [ ]:
data.loc[(data['модель'] == 'Corolla'), 'вид_топлива'] = "p"

data.loc[(data['модель'] == 'Corolla'), 'тип_привода'] = "FWD"

[Сайт DROM](https://www.drom.ru/catalog/toyota/hilux_pick_up/2019/) сообщает нам, что автомобиль Toyota HILUX DC оснащался только дизельным двигателем, имел только АКПП и только полный привод.  
Внесем изменения в датасет.

In [ ]:
data.loc[(data['модель'] == 'HILUX DC'), "бренд"].unique()

array(['Toyota'], dtype=object)

In [ ]:
data.loc[(data['модель'] == 'HILUX DC'), 'вид_топлива'] = "d"
data.loc[(data['модель'] == 'HILUX DC'), 'коробка_передач'] = "АКПП"
data.loc[(data['модель'] == 'HILUX DC'), 'тип_привода'] = "4WD"

#### Oбработаем значение nan

In [ ]:
data.loc[(data['коробка_передач'].isna())]

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013
861,2019,Март,СемАЗ,Foton,BJ3253DMPKB-AD,"Грузовой автомобиль марки ""Foton BJ3253DMPKB-A...",2019,Республика Казахстан,NaN,NaN,NaN,NaN,NaN,Алматы,Меридиан АВТО Алматы,NaN,1.0,48275.86207,48275.86207,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
862,2019,Июнь,СемАЗ,Foton,BJ3253DMPKB-AD,"Грузовой автомобиль марки ""Foton BJ3253DMPKB-A...",2018,Республика Казахстан,NaN,NaN,NaN,NaN,NaN,Усть-Каменогорск,СемАЗ,NaN,1.0,49735.91972,49735.91972,Восточно-Казахстанская область,Коммерческие автомобили,Крупнотоннажные грузовики
863,2019,Июнь,СемАЗ,Foton,BJ3253DMPKB-AD,"Грузовой автомобиль марки ""Foton BJ3253DMPKB-A...",2018,Республика Казахстан,NaN,NaN,NaN,NaN,NaN,Кокшетау,Вираж Кокшетау,NaN,1.0,48784.39304,48784.39304,Акмолинская область,Коммерческие автомобили,Крупнотоннажные грузовики
864,2019,Июль,СемАЗ,Foton,BJ3253DMPKB-AD,"Грузовой автомобиль марки ""Foton BJ3253DMPKB-A...",2018,Республика Казахстан,NaN,NaN,NaN,NaN,NaN,Алматы,Вираж Алматы,NaN,5.0,48725.42295,243627.11480,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
865,2019,Август,СемАЗ,Foton,BJ3253DMPKB-AD,"Грузовой автомобиль марки ""Foton BJ3253DMPKB-A...",2018,Республика Казахстан,NaN,NaN,NaN,NaN,NaN,Алматы,СемАЗ,NaN,2.0,31031.65229,62063.30457,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39943,2019,Сентябрь,Renault Россия,Renault,Arkana,B3P4M3TX5C,2019,Российская Федерация,NaN,NaN,NaN,NaN,NaN,Нур-Султан,Кристалл Авто Астана,NaN,1.0,21327.27747,21327.27747,г.Нур-Султан,Внедорожники,Компактные SUV
39944,2019,Сентябрь,Renault Россия,Renault,Arkana,B32M1TX5C,2019,Российская Федерация,NaN,NaN,NaN,NaN,NaN,Уральск,Урал-Кров Авто,NaN,1.0,20889.17526,20889.17526,Западно-Казахстанская область,Внедорожники,Компактные SUV
39945,2019,Сентябрь,Renault Россия,Renault,Arkana,B32M1TX5C,2019,Российская Федерация,NaN,NaN,NaN,NaN,NaN,Атырау,Урал-Кров Авто,NaN,1.0,20889.17526,20889.17526,Атырауская область,Внедорожники,Компактные SUV
39946,2019,Сентябрь,Renault Россия,Renault,Arkana,B32M1TX5C,2019,Российская Федерация,NaN,NaN,NaN,NaN,NaN,Уральск,Урал-Кров Авто,NaN,1.0,20889.17526,20889.17526,Западно-Казахстанская область,Внедорожники,Компактные SUV


In [ ]:
data.loc[(data['коробка_передач'].isna()), "модификация"].unique()

array(['Грузовой автомобиль марки "Foton BJ3253DMPKB-AD" (25 тонн)',
       'H350 BUS', 'H350 VAN 6.2', 'H350 Ambulance', nan, 'HD 160',
       'HD65 Euro 4 KZ', 'HD65 Euro 5', 'Euro 5', 'Euro 4', 'HD78 Euro 5',
       'HD78/ЕХ8', 'Luxury', 'Universe Luxury', 'D-MAX', 'Isuzu D-MAX',
       'Isuzu NMR85', 'Isuzu NPR75', '70C15', '50C15V',
       '45C15V (Автобус 14+1)', '50С15 (изотермический фургон)', '6х4',
       'Luxury 1.6 MT', 'Intelligent 1.6 CVT', 'Intelligent',
       'FL Luxury 2.0 MT5', 'FL Intelligent 2.0T MT6', 'FL Luxury',
       'FL Intelligent', 'Luxury 2.0 MT5', 'Intelligent 2.0T MT6',
       'TGS 19.440 4x2 BLS', 'TGS 26.400 6x4 BLS', 'TGS 26.360 6x4 BL',
       'TGX 18.440 4x2 BLS', 'TGS 26.360 6x4 BB', 'TGS 26.400 6x4 BLSWW',
       'MAZDA 3', 'MAZDA 6', 'MAZDA CX-5', 'Actros 1845 LS 4x2', 'V 250',
       'Active 1.6 AT', 'Active', 'Active 1.6 МТ',
       'Partner VU Active 1.6 МТ', 'Elegant МТ', 'ELEGANT AT',
       'Elegant АT', 'Optimum MT', 'Comfort Plus МТ', 'CD

In [ ]:
len(data.loc[(data['коробка_передач'].isna()), 'модель'].unique())

76

<div style="border-radius: 45px;border:solid red 7px; padding: 40px">
    
    Общее количество моделей без указания вида коробки передач составляет 76 моделей.  
    Это довольно трудозатратно, чтобы выяснять в интернете для  каждой из них вид коробки передач.  
    Отложим пока заполнение данных в столбце "коробка_передач"


#### приведем все значения к нижнему регистру

In [ ]:
# приведем все значения к нижнему регистру
data.коробка_передач = data.коробка_передач.str.lower()

####  Категоризируем тип коробки передач - разделим на `МКПП` и `АКПП`
- Всё, что содержит букву `м` на кириллице или `m` на латинице - что соответствует механической коробке передач - обозначим как `МКПП`
- всё остальное будет соответствовать автоматической или автоматизированной, роботизированной коробке передач - обозначим как `АКПП`

##### "Вытащим" при возможности тип коробки передач из столбца `модификация`.
В некоторых записях отсутствует значение короки передач в столбце `коробка_передач`. Но при этом есть указание на коробку передач в столбце `модификация`.   

"Вытащим" при возможности тип коробки передач из столбца `модификация`.  
Новые значения запишем во временный столбец `purpose_category`  
Применим функцию `categorize_purpose_second`

In [ ]:
MT = ['MT', 'M/T', 'МКП']
AT = ['AT', 'A/T']
ex = ['SEAT']
def categorize_purpose_second(total_income):
    #if data.loc[(data['коробка_передач'].isna()), 'коробка_передач'].isna():
        MT = ['MT', 'M/T', 'МКП']
        AT = ['AT', 'A/T']
        ex = ['SEAT']
        for t in MT:
            try:
                if t in total_income:
                    return 'МКПП'
            except:
                pass
        for v in AT:
            try:
                if v in ex:
                    #print (v)
                    pass
                elif v in total_income:
                    return 'AКПП'
            except:
                pass
        return


In [ ]:
data['purpose_category'] = ''
data['purpose_category'] = tqdm(data['модификация'].apply(categorize_purpose_second))
print (data['purpose_category'].isna().sum())

100%|██████████| 39966/39966 [00:00<00:00, 3623560.96it/s]

38038


In [ ]:
#data.loc[~(data['purpose_category'].isna()), 'модификация'].unique()

In [ ]:
#data.loc[~(data['purpose_category'].isna()) & (data['коробка_передач'].isna()), 'модификация'].unique()

##### Унифицируем значения из столбца `коробка_передач`.
Новые значения также запишем во временный столбец `purpose_category`   
Применим функцию `categorize_purpose`

In [ ]:
def categorize_purpose(total_income):
    try:
        if 'м' in total_income:
            return 'МКПП'
        elif 'm' in total_income:
            return 'МКПП'
        return 'AКПП'
    except:
        pass


In [ ]:
data['purpose_category'] = tqdm(data['коробка_передач'].apply(categorize_purpose))
print (data['purpose_category'].isna().sum())

100%|██████████| 39966/39966 [00:00<00:00, 3472748.16it/s]

2820


In [ ]:
data.query('purpose_category == "МКПП"')['коробка_передач'].unique()

array(['5 мт', 'mt', '5мт', 'мех.', '6 mt', '6mt', 'мкпп', 'мт', '6 мт',
       '6мт', '5mt', '6m/t', 'мt', '5m', 'm/t', 'мкп', '5м', 'м/t'],
      dtype=object)

In [ ]:
data.query('purpose_category == "AКПП"')['коробка_передач'].unique()

array(['s-tronic', 'tiptronic', 'акпп', 'акп', 'steptronic', 'at', '6 ат',
       '6ат', '8at', 'powershift s6', '6at', '6 at', '5 at', '5at',
       '7 dct', '7dct', '8 at', 'cvt', 'dct', '4 ат', '4ат', '4at', '8ат',
       '8 ат', 'cvt (вариатор)', 'cvt (вариatор)', 'cvt(вaриatор)',
       '7g-tronic', '9g-tronic', '12at', '12ат', '5ат', '7ат', '7at',
       'pdk', '8-ступ акпп', '8', '8акпп', '8 акпп tiptronic s',
       '7 акпп (pdk)', '7 акпп pdk', '8 акпп (pdk)', 'ат', 'аt', '7 dsg',
       '7dsg', '6 dsg', '6dsg', '#н/д', 'dsg', '6a', '6а', 'a/t',
       ' 7 акп', '7акп', 'а/t', '8a', '9at', 'редуктор'], dtype=object)

In [ ]:
data.query('purpose_category.isna()')['коробка_передач'].unique()

array([nan], dtype=object)

In [ ]:
data.query('коробка_передач == "#н/д"')['модель']

31148    Corolla
31149    Corolla
Name: модель, dtype: object

##### Обновим значения в столбце `коробка_передач`.
Перенесем значения из временного столбца `purpose_category` в столбец `коробка_передач`.

In [ ]:
data['коробка_передач'] = data['purpose_category']

In [ ]:
len(data.loc[(data['коробка_передач'].isna())])

2820

<div style="border-radius: 45px;border:solid red 7px; padding: 40px">
    
У нас осталось 2820 пустых значений в столбце `коробка передач`.  
Обработаем их чуть позже

### Обработка данных в столбце `тип_привода`
Выведем на экран уникальные значения, указанные в столбце `тип_привода`

In [ ]:
sorted(data.тип_привода.unique().astype(str))

['0',
 '2 WD',
 '2WD',
 '4 WD',
 '4Motion',
 '4WD',
 '4X2',
 '4X4',
 '4x2',
 '4x4',
 '4х2',
 '4х2.2',
 '4х4',
 'AWD',
 'FF',
 'FWD',
 'RWD',
 'nan',
 'quattro',
 'Астана',
 'Задний',
 'Передний',
 'Передний (FF)',
 'Полный',
 'передний',
 'полный']

"Почистим" данные:

- обработаем исключения - их проверим вручную: '0', 'Астана'
    - проверим строки, насколько соответствует отображение данных по столбцам
    - найдем марку и модель автомобиля и найдем в интернете соответствующий объем двигателя    
- nan - найдем марку и модель автомобиля и найдем в интернете соответствующий объем двигателя
- составим словарь для замены и приведения к единообразию. Обозначим:
 - FWD - передний привод
 - RWD - задний привод
 - AWD - полный привод
 - 2WD - привод на одну ось - но не известно, на какую

#### Oбработаем значение '0'

In [ ]:
data.query('тип_привода == "0"')['модель'].unique()

array(['Duster', 'Kaptur', 'Logan'], dtype=object)

Согласно [Сайта](https://www.drom.ru/catalog/renault/kaptur/specs/drive_type/) Рено Kaptur комплектовался и передним и полным приводом. Аналогиная ситуация с Рено Duster. У нас отсутствуют какие-либо данные для дополнительной идентификации.   

Рено Logan при этом выпускался только на переднем приводе. Приведем значения в соответствие.

In [ ]:
data.loc[(data.модель == "Logan"),'тип_привода'] = "FWD"

#### Oбработаем значение 'Астана'

In [ ]:
data.query('тип_привода == "Астана"')

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013,purpose_category
15771,2019,Май,ТК КАМАЗ,KAMAZ,65116,KAMAZ 65116,2019,Республика Казахстан,d,NaN,МКПП,Астана,Грузовой автомобиль,Нур-Султан,Акмолинский Автоцентр КАМАЗ,Юр. Лицо,1.0,55655.86316,55655.86316,г.Нур-Султан,Коммерческие автомобили,Крупнотоннажные грузовики,МКПП
39549,2019,Май,ТК КАМАЗ,KAMAZ,45141,KAMAZ 45141,2019,Республика Казахстан,d,NaN,МКПП,Астана,Грузовой автомобиль,Нур-Султан,Акмолинский Автоцентр КАМАЗ,Юр. Лицо,1.0,53963.81966,53963.81966,г.Нур-Султан,Коммерческие автомобили,Крупнотоннажные грузовики,МКПП


Согласно [Сайта](https://kamautocentr.ru/catalogue/Sedelnyetygachievro5/65116-7010-48-a5-163.html?utm_source=yandex&utm_medium=cpc&utm_campaign=ПОИСК%2FСпецпредложение%2F&utm_content=54901-70014-94&utm_term=---autotargeting&yclid=4598577166838661119) Официального дилера KAMAZ 65116	производится с колесной формулой 6х4 и является заднеприводным автомобилем.  

Согласно [Сайта "Грузовик"](https://gruzovik.biz/articles/kamaz-45141-tekhnicheskie-kharakteristiki) KAMAZ 45141 обладает колесной формулой 6х6 и является полноприводным автомобилем.  

Приведем значения в соответствие для всех имеющихся в датафрейме соответствующих значений.

In [ ]:
data.loc[(data.модификация == "KAMAZ 65116"),'тип_привода'] = "RWD"
data.loc[(data.модификация == "KAMAZ 45141"),'тип_привода'] = "AWD"

#### Oбработаем значение nan

In [ ]:
#data.loc[(data['тип_привода'].isna())]

In [ ]:
#data.loc[(data['тип_привода'].isna()), "модификация"].unique()

In [ ]:
len(data.loc[(data['тип_привода'].isna()), 'модель'].unique())

94

In [ ]:
len(data.loc[(data['тип_привода'].isna())])

3549

<div style="border-radius: 45px;border:solid red 7px; padding: 40px">
    
Общее количество моделей без указания типа привода составляет 94 моделей.  
Общее количество записей без указания типа привода - 3549 записей.  
Это довольно трудозатратно, чтобы выяснять в интернете для  каждой из них вид коробки передач.  
Отложим пока заполнение данных в столбце "коробка_передач"


####  Категоризируем тип привода - разделим на
 - FWD - передний привод
 - RWD - задний привод
 - AWD - полный привод
 - 2WD - привод на одну ось - но не известно, на какую`

##### "Вытащим" при возможности тип привода из столбца `модификация`.
В некоторых записях отсутствует значение короки передач в столбце `тип_привода`. Но при этом есть указание на тип привода в столбце `модификация`.   

"Вытащим" при возможности тип привода из столбца `модификация`.  
Идентифицируемые указания на тип привода: '4Motion', '4x4', '4MOTION', '4x2', '6x4'
Новые значения запишем во временный столбец `drive_category`  
Применим функцию `categorize_drive_per_modif`

In [ ]:
#FWD = [] # - передний привод
RWD_per_modif = ['4x2', '6x4'] # - задний привод
AWD_per_modif = ['4Motion', '4x4', '4MOTION'] # - полный привод
#2WD = [] # - привод на одну ось - но не известно, на какую

In [ ]:
def categorize_drive_per_modif(total_income):
    #if data.loc[(data['коробка_передач'].isna()), 'коробка_передач'].isna():
        MT = ['MT', 'M/T', 'МКП']
        AT = ['AT', 'A/T']
        ex = ['SEAT']
        for t in RWD_per_modif:
            try:
                if t in total_income:
                    return 'RWD'
            except:
                pass
        for v in AWD_per_modif:
            try:
                if v in total_income:
                    return 'AWD'
            except:
                pass
        return


In [ ]:
data['drive_category'] = ''
data['drive_category'] = tqdm(data['модификация'].apply(categorize_drive_per_modif))
print (data['drive_category'].isna().sum())

100%|██████████| 39966/39966 [00:00<00:00, 3538578.77it/s]

39799


In [ ]:
len(data.query('~drive_category.isna() and тип_привода.isna()')[['модификация', 'тип_привода', 'drive_category']])


74

<div style="border-radius: 45px;border:solid blue 7px; padding: 40px">
Мы заполнили 74 строки (из 3549 строк) значениями, содержащими тип привода - из тех, у которых в столбце `тип_привода` отсуствует значение!

##### Унифицируем значения из столбца `тип_привода`.
Новые значения также запишем во временный столбец `drive_category`   
Применим функцию `categorize_drive`

Создадим переменную `type_drive`, в которую сохраним перечень уникальных значений, присутствующих в столбце `тип_привода`

In [ ]:
type_drive = sorted(data.тип_привода.unique().astype(str))

Сформируем переменные для каждого типа привода, куда запишем соответствующие значения из столбца `тип_привода`

In [ ]:
# Номера элементов для переднего привода
drive_FWD = [14, 15, 20, 21, -2]
# формируем переменную с идентификаторами переднего привода
type_drive_FWD = []
for i in drive_FWD:
    type_drive_FWD.append(type_drive[i])

type_drive_FWD

['FF', 'FWD', 'Передний', 'Передний (FF)', 'передний']

In [ ]:
# Номера элементов для заднего привода
drive_RWD = [6, 8, 10, 11, 16, 19]
# формируем переменную с идентификаторами заднего привода
type_drive_RWD = []
for i in drive_RWD:
    type_drive_RWD.append(type_drive[i])

type_drive_RWD

['4X2', '4x2', '4х2', '4х2.2', 'RWD', 'Задний']

In [ ]:
# Номера элементов для полного привода
drive_AWD = [3, 4, 5, 7, 9, 12, 13, 18, 22, 24]
# формируем переменную с идентификаторами полного привода
type_drive_AWD = []
for i in drive_AWD:
    type_drive_AWD.append(type_drive[i])

type_drive_AWD

['4 WD',
 '4Motion',
 '4WD',
 '4X4',
 '4x4',
 '4х4',
 'AWD',
 'quattro',
 'Полный',
 'полный']

In [ ]:
# Номера элементов для привод на одну ось - но не известно, на какую
drive_2WD = [1, 2]
# формируем переменную с идентификаторами привод на одну ось - но не известно, на какую
type_drive_2WD = []
for i in drive_2WD:
    type_drive_2WD.append(type_drive[i])

type_drive_2WD

['2 WD', '2WD']

Произведем категоризацию данных о типе привода.  
Применим функцию `categorize_drive`  
Результат сохраним в столбец `drive_category`

In [ ]:
def categorize_drive(total_income):
    #if data.loc[(data['коробка_передач'].isna()), 'коробка_передач'].isna():

        for t in type_drive_FWD:
            try:
                if t in total_income:
                    return 'FWD'
            except:
                pass
        for s in type_drive_RWD:
            try:
                if s in total_income:
                    return 'RWD'
            except:
                pass
        for v in type_drive_AWD:
            try:
                if v in total_income:
                    return 'AWD'
            except:
                pass
        for w in type_drive_2WD:
            try:
                if w in total_income:
                    return '2WD'
            except:
                pass
        return


In [ ]:
data['drive_category'] = ''
data['drive_category'] = tqdm(data['тип_привода'].apply(categorize_drive))
print (data['drive_category'].isna().sum())

100%|██████████| 39966/39966 [00:00<00:00, 3478224.54it/s]

3609


##### Обновим значения в столбце `тип_привода`.
Перенесем значения из временного столбца `drive_category` в столбец `тип_привода`.

In [ ]:
data['тип_привода'] = data['drive_category']

In [ ]:
len(data.loc[(data['тип_привода'].isna())])

3609

<div style="border-radius: 45px;border:solid blue 7px; padding: 40px">
    
У нас осталось 3609 пустых значений в столбце `тип_привода`.  
Обработаем их чуть позже

<div style="border-radius: 45px;border:solid blue 7px; padding: 40px">
    
Подведем промежуточный итог и посмотрим, какая доля ячеек осталась незаполненными

In [ ]:
display(round(data.isna().mean() * 100).astype(int))

год                                0
месяц                              0
компания                           0
бренд                              0
модель                             0
модификация                        9
год_выпуска                        1
страна_производитель               0
вид_топлива                        7
объём_двиг_л                      10
коробка_передач                    7
тип_привода                        9
сегмент                           17
регион                             0
наименование_дилерского_центра     0
тип_клиента                       18
количество                         0
цена_usd                           0
продажа_usd                        0
область                            0
сегментация_2013                   0
класс_2013                         0
purpose_category                   7
drive_category                     9
dtype: int64

### Заполним модой значения в столбцах
- 'вид_топлива',
- 'объём_двиг_л',
- 'коробка_передач',
- 'тип_привода'

Заполним на основе значений в столбце `модель`.

Используем функцию `fill_gaps`

In [ ]:
def fill_gaps(columb_name, support_col):
    variable_name = data.loc[(data[columb_name].isna()), support_col].unique().tolist()
    print("\033[94m""\033[1m""На входе отсутствуют данные для вычисления моды у моделей:""\033[0m")
    print(variable_name)

    x_temp = []
    for i in variable_name:
        try:
            data.loc[(data[support_col] == i) & (data[columb_name].isna()), columb_name] = \
                data.loc[(data[support_col] == i), columb_name].mode()[0]

        except:
            x_temp.append(i)

    print("\033[94m""\033[1m""После заполнения отсутствуют данные для вычисления моды у моделей:""\033[0m")
    print(x_temp)
    print("\033[94m""\033[1m""Заполнили", len(variable_name) - len(x_temp),
          "моделей, что составило", (1-len(x_temp)/len(variable_name))*100, '%' "\033[0m")

In [ ]:
cols_names_4 = ['вид_топлива', 'объём_двиг_л','коробка_передач', 'тип_привода']

In [ ]:
for i in cols_names_4:
    fill_gaps(i, 'модель')

На входе отсутствуют данные для вычисления моды у моделей:
['BJ3253DMPKB-AD', '2310', '2705', '2752', '3221', '3302', '3308', 'Next', 'H350', 'HD 160', 'HD 65', 'HD 78', 'Universe', 'Daily', 'Daily A', 'Daily V', 'Stralis', 'Tipper', 'S3', 'S5', 'XJ', 'ВИС', 'Discovery Sport', 'Range Rover', 'Range Rover Evoque', 'Range Rover Sport', 'TG', '3', '6', 'CX-5', '301', 'Partner Panel Van', 'Duster', 'Kaptur', 'Koleos', 'Logan', 'Master', '3151', '3741', 'Cargo', 'Patriot Pickup', 'Amarok', 'Caravelle', 'Polo', 'Tiguan', 'Touareg', 'Transporter Kasten', 'XC90', '4320', '4320-1951-40', '4320-1951-60', '5557', 'CX-9', 'HD 35', 'Range Rover Velar', 'E-PACE', 'Dokker', 'N120', 'N75', 'T6', 'SX3258DR384', 'XC60', 'N56', 'BJ1069VDJEA-F1', 'BWC6665GA5', 'HF-D105', 'Teramont', 'I-Pace', '32551', 'EX8', 'HFF6850G', 'Arkana']
После заполнения отсутствуют данные для вычисления моды у моделей:
['BJ3253DMPKB-AD', 'HD 160', 'Daily A', 'Daily V', 'Stralis', 'Tipper', '3', '6', 'CX-5', 'Kaptur', 'Koleos', '

<div style="border-radius: 45px;border:solid blue 7px; padding: 40px">
Как видим, остались незаполненные значения

### Еще раз заполним модой значения в столбцах
- 'вид_топлива',
- 'объём_двиг_л',
- 'коробка_передач',
- 'тип_привода'

На этот раз либо заполним на основе значений в столбце `класс_2013`.

Используем функцию `fill_gaps_per_class`

In [ ]:
for i in cols_names_4:
    fill_gaps(i, 'класс_2013')

На входе отсутствуют данные для вычисления моды у моделей:
['Крупнотоннажные грузовики', 'Среднетоннажные грузовики', 'Микроавтобусы', 'Развозные автомобили', 'Малотоннажные грузовики', 'C класс', 'D класс', 'Компактные SUV', 'Субкомпактные SUV', 'B класс', 'Полноразмерные SUV', 'Компактвэн', 'Средние автобусы', 'Большие автобусы']
После заполнения отсутствуют данные для вычисления моды у моделей:
[]
Заполнили 14 моделей, что составило 100.0 %
На входе отсутствуют данные для вычисления моды у моделей:
['Крупнотоннажные грузовики', 'Среднетоннажные грузовики', 'Микроавтобусы', 'Развозные автомобили', 'Малотоннажные грузовики', 'C класс', 'D класс', 'Компактные SUV', 'Субкомпактные SUV', 'B класс', 'Полноразмерные SUV', 'Компактвэн', 'Средние автобусы', 'Большие автобусы']
После заполнения отсутствуют данные для вычисления моды у моделей:
[]
Заполнили 14 моделей, что составило 100.0 %
На входе отсутствуют данные для вычисления моды у моделей:
['Крупнотоннажные грузовики', 'Среднетоннажны

Удостоверимся, что в столбцах 'вид_топлива', 'объём_двиг_л','коробка_передач', 'тип_привода' заполнены все значения:

In [ ]:
display(round(data.isna().mean() * 100).astype(int))

год                                0
месяц                              0
компания                           0
бренд                              0
модель                             0
модификация                        9
год_выпуска                        1
страна_производитель               0
вид_топлива                        0
объём_двиг_л                       0
коробка_передач                    0
тип_привода                        0
сегмент                           17
регион                             0
наименование_дилерского_центра     0
тип_клиента                       18
количество                         0
цена_usd                           0
продажа_usd                        0
область                            0
сегментация_2013                   0
класс_2013                         0
purpose_category                   7
drive_category                     9
dtype: int64

#### категоризируем данные в столбце `тип_клиента` и `количество`
Выведем на экран уникальные значения, указанные в столбце `тип_клиента`

### Обработка данных в столбцах `тип_клиента` и `количество`
Выведем на экран уникальные значения, указанные в столбце `тип_клиента`

In [ ]:
sorted(data.тип_клиента.unique().astype(str))

['nan',
 'Корп. клиент',
 'Физ. Лицо',
 'ФизЛицо',
 'Юр. Лицо',
 'Юр.Лицо',
 'ЮрЛицо',
 'физ.лицо',
 'физическое',
 'юридическое']

In [ ]:
# приведем все значения к нижнему регистру
data.тип_клиента = data.тип_клиента.str.lower()

#### Унифицируем значения из столбца `тип_клиента`.
Применим функцию `categorize_client_type`

In [ ]:
def categorize_client_type(total_income):
    try:
        if 'физ' in total_income:
            return 'Физлицо'
        elif 'юр' in total_income:
            return 'Юрлицо'
        elif 'корп' in total_income:
            return 'Юрлицо'

    except:
        pass


In [ ]:
data['тип_клиента'] = data['тип_клиента'].apply(categorize_client_type)
print (data['тип_клиента'].isna().sum())

7047


In [ ]:
sorted(data.тип_клиента.unique().astype(str))

['None', 'Физлицо', 'Юрлицо']

#### Oбработаем значение nan

In [ ]:
#data.loc[(data['тип_клиента'].isna())]

In [ ]:
data.loc[(data['тип_клиента'].isna()), "количество"].unique()

array([  2.,   1.,   4.,   3.,   7.,   5.,   8.,  10.,   6.,   9.,  11.,
        19.,  12.,  18.,  37.,  17.,  15.,  24.,  29.,  14.,  13.,  25.,
        63.,  23.,  30.,  21.,  60.,  nan,  22.,  16.,  35.,  28.,  32.,
        54., 115.,  46.,  33.,  47.,  20.,  66.,  27.,  50.,  36.,  26.,
        31.,  34.,  40.,  43.,  51.,  42.,  38.,  41., 100.,  -1.,  79.,
        62.])

In [ ]:
len(data.loc[(data['тип_клиента'].isna()), "количество"].unique())

56

In [ ]:
len(data.loc[(data['тип_клиента'].isna())])

7047

- Не заполнено 7047 записей.  
- Клиенты покупали от 1 до 115 единиц автомобилей.  
- Данные представлены в числовом виде.   
- Исключения - -1 (минус 1) и nan - их рассмотрим в первую очередь.
- Примем для категшоризации, что если клиент приорел один или 2 автомобиля в одной сделке - это физлицо, если более двух - автомобилей в одной сделке - это юрлицо.

In [ ]:
data.loc[(data['тип_клиента'].isna()) & (data['количество'] == -1)]

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013,purpose_category,drive_category
25728,2019,Август,БИПЭК АВТО,Skoda,Superb,NaN,2019,Республика Казахстан,p,2.0,AКПП,AWD,E класс,Тараз,БИПЭК АВТО Тараз,None,-1.0,35588.25,-35588.25,Жамбылская область,Легковые автомобили,D класс,AКПП,AWD


Вероятно, это ошибка при заполнении и клиент приобрел 1 автомобиль. Исправим ошибку

In [ ]:
data.loc[(data['тип_клиента'].isna()) & (data['количество'] == -1), 'количество'] = 1

In [ ]:
data.loc[(data['тип_клиента'].isna()) & (data['количество'].isna())]

,год,месяц,компания,бренд,модель,модификация,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,сегмент,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013,purpose_category,drive_category
16741,2019,Август,БИПЭК АВТО,Kia,Sportage,NaN,2019,Республика Казахстан,p,2.0,AКПП,AWD,Компактные SUV,Караганда,БИПЭК АВТО Караганда,None,NaN,20700.0,0.0,Карагандинская область,Внедорожники,Компактные SUV,AКПП,AWD
17368,2019,Август,БИПЭК АВТО,Lada,Granta,NaN,2019,Республика Казахстан,p,1.6,AКПП,FWD,B класс,Алматы,БИПЭК АВТО Алматы,None,NaN,7100.0,0.0,г.Алматы,Легковые автомобили,B класс,AКПП,FWD
18676,2019,Август,БИПЭК АВТО,Lada,XRAY,NaN,2018,Республика Казахстан,p,1.6,МКПП,FWD,B класс,Экибастуз,БИПЭК АВТО Экибастуз,None,NaN,13800.0,0.0,Павлодарская область,Легковые автомобили,B класс,МКПП,FWD
37416,2019,Август,БИПЭК АВТО,UAZ,3909,NaN,2018,Республика Казахстан,p,2.7,МКПП,AWD,Развозные автомобили,Алматы,БИПЭК АВТО Алматы,None,NaN,11200.0,0.0,г.Алматы,Коммерческие автомобили,Развозные автомобили,МКПП,AWD
37727,2019,Август,БИПЭК АВТО,UAZ,Patriot,NaN,2018,Республика Казахстан,p,2.7,МКПП,AWD,Полноразмерные SUV,Семей,БИПЭК АВТО Семей,None,NaN,13800.0,0.0,Восточно-Казахстанская область,Внедорожники,Среднеразмерные SUV,МКПП,AWD
38775,2019,Август,БИПЭК АВТО,Skoda,Kodiaq,NaN,2019,Республика Казахстан,p,2.0,AКПП,AWD,Стандартные SUV,Нур-Султан,БИПЭК АВТО Астана,None,NaN,40000.0,0.0,г.Нур-Султан,Внедорожники,Полноразмерные SUV,AКПП,AWD


- 6 различных автомобилей
- проданы в разных регионах
- проданы одной компанией
- для решения обратимся к цене: сравним цену конкретной сделки со средней ценой на автомобиль этой марки

In [ ]:
# создадим список моделей
roll_models = data.loc[(data['тип_клиента'].isna()) & (data['количество'].isna()), 'модель'].tolist()

In [ ]:
# список индексов для последующей проверки
roll_index = data.loc[(data['тип_клиента'].isna()) & (data['количество'].isna())].index.tolist()
roll_index

[16741, 17368, 18676, 37416, 37727, 38775]

In [ ]:
for i in data.loc[(data['тип_клиента'].isna()) & (data['количество'].isna()), 'модель'].tolist():

    # определим переменную t как отношение стоимости продажи в сделке к модальной величине цены данной модели
    # Округлим t в меньшую сторону
    t = math.floor((data.loc[(data['модель'] == i) , 'продажа_usd'].mean() / \
        data.loc[(data['тип_клиента'].isna()) & (data['количество'].isna()) & (data['модель'] == i), 'цена_usd']\
        .mode()[0]))#.astype(int)
    print(t)
    # подставим значение t в пропущенные ячейки количества проданных автомашин
    data.loc[(data['тип_клиента'].isna()) & (data['количество'].isna()) & (data['модель'] == i), 'количество'] = t
    '''if  t < 2:
        print("\033[94m""Автомобиль", i, 'продан в 1 экземпляре,'"\033[1m" ' клиент - физлицо'"\033[0m’")
    elif t < 3:
        print("\033[94m""Автомобиль", i, 'продан в 2 экземплярах,'"\033[1m" ' клиент - физлицо'"\033[0m’")
    else:
        print("\033[93m""Автомобиль", i, 'продан БОЛЕЕ, чем в 2-х экземпляре,'"\033[1m"' клиент - юрлицо' '\033[0m')
        '''

2
8
1
1
1
1


Проверим

In [ ]:
for i in roll_index:
    #display(data.query('index == @i')['количество'][0])
    display(data.loc[(data.index == i),'количество'])

16741    2.0
Name: количество, dtype: float64

17368    8.0
Name: количество, dtype: float64

18676    1.0
Name: количество, dtype: float64

37416    1.0
Name: количество, dtype: float64

37727    1.0
Name: количество, dtype: float64

38775    1.0
Name: количество, dtype: float64

#### Обработаем пропущенные значения в столбце `тип_клиента`
Для пропущенных значений установим `тип_клиента`:
- если в строке `количество` равно 1 или 2 -присвоим `тип_клиента` - Физлицо
- если в строке `количество` более 2 -присвоим `тип_клиента` - Юрлицо

In [ ]:
data.loc[(data['тип_клиента'].isna()) & (data['количество'] <= 2), 'тип_клиента'] = 'Физлицо'
data.loc[(data['тип_клиента'].isna()) & (data['количество'] > 2), 'тип_клиента'] = 'Юрлицо'

### Обработка данных в столбце `наименование_дилерского_центра`
Выведем на экран уникальные значения, указанные в столбце `наименование_дилерского_центра`

In [ ]:
(data.наименование_дилерского_центра.unique().astype(str))

array(['Mercur Auto Алматы', 'Mercur Auto Астана',
       'Автоцентр-Бавария Астана', 'Автоцентр-Бавария Алматы',
       'Автоцентр Бавария Восток', 'ЖАЙЫК HYUNDAI MOTORS',
       'Hyundai Auto Kostanai', 'БИПЭК АВТО Костанай',
       'БИПЭК АВТО Астана', 'БИПЭК АВТО Павлодар', 'БИПЭК АВТО Алматы',
       'БИПЭК АВТО Семей', 'БИПЭК АВТО Шымкент', 'БИПЭК АВТО Актау',
       'БИПЭК АВТО Усть-Каменогорск', 'БИПЭК АВТО Атырау',
       'БИПЭК АВТО Актобе', 'БИПЭК АВТО Уральск', 'БИПЭК АВТО Караганда',
       'БИПЭК АВТО Петропавловск', 'Вираж Алматы', 'Вираж Кокшетау',
       'Вираж Уральск', 'Вираж Петропавловск', 'Вираж Павлодар',
       'Вираж Караганда', 'Вираж Астана', 'Вираж Костанай', 'Вираж Актау',
       'Вираж Усть-Каменогорск', 'Вираж Актобе', 'Вираж Атырау',
       'Вираж Шымкент', 'Вираж Семей', 'Вираж Рудный', 'Вираж Кызылорда',
       'Вираж Тараз', 'Cadillac Almaty', 'Daewoo Bus Kazakhstan',
       'Авто Сити', 'Ford Атырау', 'Меридиан АВТО Алматы', 'СемАЗ',
       'Меридиан

Дублируются названия двух дилерских центров:

- 'Nissan Center Уральск' и  'Nissan Centr Уральск'
- 'Auto Trader ' и  'Auto trader'

Исправим

In [ ]:
data.loc[(data['наименование_дилерского_центра'] == "Nissan Centr Уральск"), 'наименование_дилерского_центра'] = \
    "Nissan Center Уральск"
data.loc[(data['наименование_дилерского_центра'] == "Auto trader"), 'наименование_дилерского_центра'] = \
    "Auto Trader "
data.loc[(data['наименование_дилерского_центра'] == "Auto Trader "), 'наименование_дилерского_центра'] = \
    "Auto Trader"

### Проверка по итогу обработки данных


Удалим созданные ранее временные столбцы:

In [ ]:
colms_1 = data.columns
# Номера столбцов под удаление
colms_to_delete = [-2,-1]

In [ ]:
for i in colms_to_delete:
    data = data.drop(colms_1[i], axis=1)
#data.columns

In [ ]:
data.columns

Index(['год', 'месяц', 'компания', 'бренд', 'модель', 'модификация',
       'год_выпуска', 'страна_производитель', 'вид_топлива', 'объём_двиг_л',
       'коробка_передач', 'тип_привода', 'сегмент', 'регион',
       'наименование_дилерского_центра', 'тип_клиента', 'количество',
       'цена_usd', 'продажа_usd', 'область', 'сегментация_2013', 'класс_2013'],
      dtype='object')

Проверим долю пустых значений

In [ ]:
display(round(data.isna().mean() * 100).astype(int))

год                                0
месяц                              0
компания                           0
бренд                              0
модель                             0
модификация                        9
год_выпуска                        1
страна_производитель               0
вид_топлива                        0
объём_двиг_л                       0
коробка_передач                    0
тип_привода                        0
сегмент                           17
регион                             0
наименование_дилерского_центра     0
тип_клиента                        0
количество                         0
цена_usd                           0
продажа_usd                        0
область                            0
сегментация_2013                   0
класс_2013                         0
dtype: int64

In [ ]:
data.columns

Index(['год', 'месяц', 'компания', 'бренд', 'модель', 'модификация',
       'год_выпуска', 'страна_производитель', 'вид_топлива', 'объём_двиг_л',
       'коробка_передач', 'тип_привода', 'сегмент', 'регион',
       'наименование_дилерского_центра', 'тип_клиента', 'количество',
       'цена_usd', 'продажа_usd', 'область', 'сегментация_2013', 'класс_2013'],
      dtype='object')

Пустые (пропушенные) значения остались в столбцах 'модификация', 'год_выпуска', 'сегмент'.  
- из столбца 'модификация' мы уже получили возможные данные по типу привода и коробке передач и модем удалить этот столбец
- столбец 'сегмент' содержит очень много уникальных значений. Информация в столбце 'класс_2013' менее подробная, но содержит достаточно значений для категоризации. Можем удалить столбец 'сегмент'
- столбец 'год_выпуска' содержит 1% пропущенных значений У нас нет осеований для его заполнения Оставим значения в нем как есть

Удалим столбцы 'модификация', 'сегмент'

In [ ]:
colms_1

Index(['год', 'месяц', 'компания', 'бренд', 'модель', 'модификация',
       'год_выпуска', 'страна_производитель', 'вид_топлива', 'объём_двиг_л',
       'коробка_передач', 'тип_привода', 'сегмент', 'регион',
       'наименование_дилерского_центра', 'тип_клиента', 'количество',
       'цена_usd', 'продажа_usd', 'область', 'сегментация_2013', 'класс_2013',
       'purpose_category', 'drive_category'],
      dtype='object')

In [ ]:
# Номера столбцов под удаление
colms_to_delete_2 = [5, 12]

In [ ]:
for i in colms_to_delete_2:
    data = data.drop(colms_1[i], axis=1)
#data.columns

### Сегментация по объему двигателя


Согласно [Большой российской энциклопедии](https://old.bigenc.ru/technology_and_technique/text/2136799) в России существует классификация легковых автомобилей в зависимости от объема двигателя.  Выглядит она нижеследующим образом. К таблице для экономии места добавим столбец со значениями от 1 до 4, соответствующие классу двигателя:

|Класс	|Объём двигателя, дм3 (литров)|Цифровое обозначение|
|:---|:---:|:---:|
Малый	|До 1,2 |1
Средний |От 1,2 до 1,8 |2
Большой |От 1,8 до 3,5 |3
Высший	|Свыше 3,5 |4

 В исследуемом датафрейме отнесем к легковым автомобилям все уникальные значения столбца `сегментация_2013` за исключение значения 'Коммерческие автомобили'

Для грузового транспорта интернет не выдает данные по заппросу о классификации по объему двигателя. Произведем такую классификацию самостоятельно и разобьем данные на три пропорциональные категории:

|Класс	|Объём двигателя, дм3 (литров)|Цифровое обозначение|
|:---|:---:|:---:|
Мал_груз	|До 2,5 | 11
Сред_груз |От 2,5 до 4| 12
Бол_груз |От 4 до 6 | 13
Выс_груз |От 6 | 14


In [ ]:
data.loc[(data['сегментация_2013'] != "Коммерческие автомобили") &\
            (data['объём_двиг_л'] > 0) & (data['объём_двиг_л'] < 1.2), 'класс_двигателя'] = 1
data.loc[(data['сегментация_2013'] != "Коммерческие автомобили") &\
            (data['объём_двиг_л'] >= 1.2) & (data['объём_двиг_л'] < 1.8), 'класс_двигателя'] = 2
data.loc[(data['сегментация_2013'] != "Коммерческие автомобили") &\
            (data['объём_двиг_л'] >= 1.8) & (data['объём_двиг_л'] < 3.5), 'класс_двигателя'] = 3
data.loc[(data['сегментация_2013'] != "Коммерческие автомобили") &\
            (data['объём_двиг_л'] >= 3.5), 'класс_двигателя'] = 4

In [ ]:
data.loc[(data['сегментация_2013'] == "Коммерческие автомобили") &\
            (data['объём_двиг_л'] > 0) & (data['объём_двиг_л'] < 2.5), 'класс_двигателя'] = 11
data.loc[(data['сегментация_2013'] == "Коммерческие автомобили") &\
            (data['объём_двиг_л'] >= 2.5) & (data['объём_двиг_л'] < 4), 'класс_двигателя'] = 12
data.loc[(data['сегментация_2013'] == "Коммерческие автомобили") &\
            (data['объём_двиг_л'] >= 4) & (data['объём_двиг_л'] < 6), 'класс_двигателя'] = 13
data.loc[(data['сегментация_2013'] == "Коммерческие автомобили") &\
            (data['объём_двиг_л'] >= 6), 'класс_двигателя'] = 14

In [ ]:
data.вид_топлива.unique()

array(['p', 'd', 'g', 'e'], dtype=object)

### Приведем названия стран к буквенному коду
Для повышения скорости обработки данных приведем названия стран к буквенному коду [Общероссийского классификатора стран мира](https://classifikators.ru/oksm#download)


In [ ]:
#oksm = pd.read_excel('oksm.xlsx', header = [6])

In [ ]:
oksm = pd.read_html('https://classifikators.ru/oksm#download', header = 1)[0]

In [ ]:
oksm.head(10)

,№,Код,Краткое,Полное,Альфа-2,Альфа-3
0,1,4,АФГАНИСТАН,Переходное Исламское Государство Афганистан,AF,AFG
1,2,8,АЛБАНИЯ,Республика Албания,AL,ALB
2,3,10,АНТАРКТИДА,NaN,AQ,ATA
3,4,12,АЛЖИР,Алжирская Народная Демократическая Республика,DZ,DZA
4,5,16,АМЕРИКАНСКОЕ САМОА,NaN,AS,ASM
5,6,20,АНДОРРА,Княжество Андорра,AD,AND
6,7,24,АНГОЛА,Республика Ангола,AO,AGO
7,8,28,АНТИГУА И БАРБУДА,NaN,AG,ATG
8,9,31,АЗЕРБАЙДЖАН,Республика Азербайджан,AZ,AZE
9,10,32,АРГЕНТИНА,Аргентинская Республика,AR,ARG


In [ ]:
oksm['Краткое'] = oksm['Краткое'].str.lower()
oksm['Полное'] = oksm['Полное'].str.lower()

In [ ]:
data['страна_производитель'].unique()

array(['Германия', 'США', 'Австрия', 'Республика Казахстан',
       'Российская Федерация', 'Корея', 'Япония', 'Таиланд', 'Китай',
       'UK', 'Узбекистан', 'Венгрия', 'Турция', 'Испания', 'Нидерланды',
       'Польша', 'Швеция', 'Белоруссия', 'Бельгия'], dtype=object)

Зададим списки для сопоставления.  
kod_roll - список названий стран, приведенных к нижнему регистру, из столбца 'Краткое название' датафрейма `oksm`.  

old_roll - список названий стран, приведенных к нижнему регистру и с исключением слова `республика`, из столбца 'страна_производитель' датафрейма `data`

In [ ]:
kod_roll = oksm['Краткое'].str.lower().tolist()
old_roll = data.страна_производитель.str.lower().str.replace('республика ', '').unique()

len(old_roll)

19

Для сохранения кодового обозначения страны создадим столбец `код_страны`. Занесем в него значения из столбца `страна_производитель`

In [ ]:
data['код_страны'] = data['страна_производитель']

Для удобства сравнения приведем названия стран в столбце `страна_производитель` к нижнему регистру. Также удалим из названий стран слово `республика`

In [ ]:
data.страна_производитель = data.страна_производитель.str.lower().str.replace('республика ', '')

In [ ]:
data.код_страны.unique()

array(['Германия', 'США', 'Австрия', 'Республика Казахстан',
       'Российская Федерация', 'Корея', 'Япония', 'Таиланд', 'Китай',
       'UK', 'Узбекистан', 'Венгрия', 'Турция', 'Испания', 'Нидерланды',
       'Польша', 'Швеция', 'Белоруссия', 'Бельгия'], dtype=object)

Обратим внимание, что в списке кодов стран уже пристуствует один код  - это `UK`.

Сопоставим названия в списках  `old_roll` и `kod_roll`.  
При выявлении совпадения присвоим символьное обозначение для страны в колонку `код_страны` датафрейма `data`

In [ ]:
sec_roll =[]

for i in old_roll:
    if i in kod_roll:
        n = oksm.loc[(oksm['Краткое'] == i), 'Альфа-2'].tolist()[0]
        data.loc[(data['страна_производитель'] == i), 'код_страны'] = n
        sec_roll.append(n)
    elif i in oksm['Полное'].str.lower().tolist():
        m = oksm.loc[(oksm['Полное'] == i), 'Альфа-2'].tolist()[0]
        data.loc[(data['страна_производитель'] == i), 'код_страны'] = m
        sec_roll.append(m)
print(sec_roll)
print(len(sec_roll))

['DE', 'AT', 'KZ', 'RU', 'JP', 'TH', 'CN', 'UZ', 'HU', 'TR', 'ES', 'PL', 'SE', 'BE']
14


In [ ]:
data['страна_производитель'].unique()

array(['германия', 'сша', 'австрия', 'казахстан', 'российская федерация',
       'корея', 'япония', 'таиланд', 'китай', 'uk', 'узбекистан',
       'венгрия', 'турция', 'испания', 'нидерланды', 'польша', 'швеция',
       'белоруссия', 'бельгия'], dtype=object)

In [ ]:
data.код_страны.unique()

array(['DE', 'США', 'AT', 'KZ', 'RU', 'Корея', 'JP', 'TH', 'CN', 'UK',
       'UZ', 'HU', 'TR', 'ES', 'Нидерланды', 'PL', 'SE', 'Белоруссия',
       'BE'], dtype=object)

Составим список `fogotten_contr` - страны, код которых не был идентифицирован.

In [ ]:
fogotten_contr = []
for i in data.код_страны.unique():
    if i not in sec_roll:
        print(i)
        fogotten_contr.append(i)
fogotten_contr


США
Корея
UK
Нидерланды
Белоруссия


['США', 'Корея', 'UK', 'Нидерланды', 'Белоруссия']

Исключим из списка `UK` - этот код был изначально, его не нужно идентифицировать дополнительно.

In [ ]:
fogotten_contr.pop(2)

'UK'

(Вновь) приведем к нижнему регистру элементы списка `fogotten_contr`

In [ ]:
fogotten_contr = list(map(str.lower,fogotten_contr))

In [ ]:
fogotten_contr

['сша', 'корея', 'нидерланды', 'белоруссия']

Найдем элементы в колонке 'Краткое название' датафрейма `oksm`, содержащие символы `кор` (от слова `Корея`) и  `нид` ( от слова `Нидерланды`

In [ ]:
roll_4 = []
for i in range(len(oksm['Краткое'].tolist())):
    if "кор" in oksm['Краткое'].tolist()[i]:
    #n = oksm.loc[(oksm['Краткое название'] == i), 'Альфа-2'].tolist()[0]
    #print(data.loc[(data['страна_производитель'] == i), 'код_страны'])
    #data.loc[(data['страна_производитель'] == i), 'код_страны'] = n
        #print(oksm['Краткое название'].tolist()[i])
        roll_4.append(oksm['Краткое'].tolist()[i])
roll_4

['корея, народно-демократическая республика',
 'корея, республика',
 'нидерланды, королевство',
 'соединенное королевство великобритании и северной ирландии']

Составим словарь для замены названий стран , состоящих в списке `fogotten_contr`

In [ ]:
fogotten_contr_dic = {'сша':'соединенные штаты',  'корея': roll_4[1], 'нидерланды': roll_4[2], 'белоруссия': 'беларусь'}
fogotten_contr_dic

{'сша': 'соединенные штаты',
 'корея': 'корея, республика',
 'нидерланды': 'нидерланды, королевство',
 'белоруссия': 'беларусь'}

In [ ]:
data['страна_производитель'].unique()

array(['германия', 'сша', 'австрия', 'казахстан', 'российская федерация',
       'корея', 'япония', 'таиланд', 'китай', 'uk', 'узбекистан',
       'венгрия', 'турция', 'испания', 'нидерланды', 'польша', 'швеция',
       'белоруссия', 'бельгия'], dtype=object)

In [ ]:
# Призведем замену названий оставшихся стран согласно словаря fogotten_contr_dic
for i in fogotten_contr:
    print(i)
    data.loc[(data['страна_производитель'] == i), 'страна_производитель'] = data['страна_производитель'].map(fogotten_contr_dic)
    #data['страна_производитель'] = data['страна_производитель'].map(fogotten_contr_dic)

сша
корея
нидерланды
белоруссия


In [ ]:
rolling = ['соединенные штаты', roll_4[1], roll_4[2], 'беларусь']

In [ ]:
for i in data.страна_производитель.unique():
    if i in kod_roll:
        n = oksm.loc[(oksm['Краткое'] == i), 'Альфа-2'].tolist()[0]
        data.loc[(data['страна_производитель'] == i), 'код_страны'] = n
        #sec_roll.append(n)
print(sorted(sec_roll) )

100%|██████████| 19/19 [00:00<00:00, 330.13it/s]

['AT', 'BE', 'CN', 'DE', 'ES', 'HU', 'JP', 'KZ', 'PL', 'RU', 'SE', 'TH', 'TR', 'UZ']


Проверим полноту символов в клонке и количество уникальных символов

In [ ]:
data.код_страны.unique()

array(['DE', 'US', 'AT', 'KZ', 'RU', 'KR', 'JP', 'TH', 'CN', 'UK', 'UZ',
       'HU', 'TR', 'ES', 'NL', 'PL', 'SE', 'BY', 'BE'], dtype=object)

In [ ]:
len(data.код_страны.unique())

19

Пропущенные значения отсутствуют. Количество символов соответствует количеству стран

<div style="border-radius: 45px;border:solid blue 7px; padding: 40px">
    
На этом приведение названий стран к буквенному коду завершено
    

### Создадим столбец, объединяющую номер месяца и год

Создадим словарь `dict_monthes`

In [ ]:
dict_monthes = {
    'Январь':1,
'Февраль':2,
'Март':3,
'Апрель':4,
'Май':5,
'Июнь':6,
'Июль':7,
'Август':8,
'Сентябрь':9,
'Октябрь':10,
'Ноябрь':11,
'Декабрь':12
}

Произведем замену

In [ ]:
data['месяц'] = data['месяц'].map(dict_monthes)

Объединим месяц и год в новом столбце `дата` и преобразуем его в формат дата-время `datetime`

In [ ]:
data['дата'] = data['месяц'].map(str) + '-' + data['год'].map(str)
data['дата'] = pd.to_datetime(data['дата'])

### Устраним дубликаты в названиях компаний

Посмотрим названия компаний и увидим ряд дубликатов.  
Сведем дубликаты в список `duplicated_comp`.   
Для заменя дубликатов создадим словарь `dict_comp_names`

In [ ]:
(data.компания.unique())

array(['Mercur Auto', 'Автоцентр-Бавария', 'БИПЭК АВТО', 'Вираж',
       'Eurasia Motor Premium', 'Daewoo Bus Kazakhstan', 'Caspian Motors',
       'Каспиан Моторс', 'СемАЗ', 'Astana Motors', 'Astana Motors ',
       'Hyundai Com Trans Kazakhstan ', 'Nissan Manufacturing RUS',
       'СВС-ТРАНС', 'Allur Auto', 'Almaty Motors Premium', 'TERRA MOTORS',
       'ТК КАМАЗ', 'Toyota Motor Kazakhstan', 'Лифан Моторс Рус',
       'MAN Truck & Bus Kazakhstan', 'Автомир ГК', 'Autokapital',
       'Автокапитал', 'ММС Рус', 'MMC RUS', 'Mercur Autos',
       'УзАвто-Казахстан', 'Равон Моторс Казахстан',
       'Ravon Motors Kazakstan', 'Renault Россия', 'Scania Central Asia',
       'Subaru Kazakhstan', 'Volkswagen Group Rus', 'Автодом Motors KST',
       'Автомир-Центр', 'ТОО "Eurasia Motor Zhaik"',
       'Scandinavian Motors', 'Хино Моторс Казахстан', 'Hino Motors '],
      dtype=object)

In [ ]:
duplicated_comp = [
        'Astana Motors ','Hino Motors ','Mercur Autos','Каспиан Моторс','ММС Рус','Autokapital','Равон Моторс Казахстан'
]

In [ ]:
dict_comp_names = {
    'Astana Motors ': 'Astana Motors',
'Hino Motors ':'Хино Моторс Казахстан',
'Mercur Autos':'Mercur Auto',
'Каспиан Моторс':'Caspian Motors',
'ММС Рус': 'MMC RUS',
'Autokapital':'Автокапитал',
'Равон Моторс Казахстан':'Ravon Motors Kazakstan'
}

In [ ]:
# Призведем замену дубликатов компаний
for i in duplicated_comp:
    #print(i)
    data.loc[(data['компания'] == i), 'компания'] = \
        data['компания'].map(dict_comp_names)


In [ ]:
data

,год,месяц,компания,бренд,модель,год_выпуска,страна_производитель,вид_топлива,объём_двиг_л,коробка_передач,тип_привода,регион,наименование_дилерского_центра,тип_клиента,количество,цена_usd,продажа_usd,область,сегментация_2013,класс_2013,класс_двигателя,код_страны,дата
0,2019,5,Mercur Auto,Audi,A3,2018,германия,p,1.4,AКПП,FWD,Алматы,Mercur Auto Алматы,Физлицо,1.0,28115.00000,28115.00000,г.Алматы,Легковые автомобили,C класс,2.0,DE,2019-05-01
1,2019,8,Mercur Auto,Audi,A3,2018,германия,p,1.4,AКПП,FWD,Алматы,Mercur Auto Алматы,Юрлицо,1.0,32246.99000,32246.99000,г.Алматы,Легковые автомобили,C класс,2.0,DE,2019-08-01
2,2019,4,Mercur Auto,Audi,A4,2018,германия,p,1.4,AКПП,FWD,Алматы,Mercur Auto Алматы,Физлицо,1.0,32000.00000,32000.00000,г.Алматы,Легковые автомобили,D класс,2.0,DE,2019-04-01
3,2019,7,Mercur Auto,Audi,A4,2018,германия,p,1.4,AКПП,FWD,Алматы,Mercur Auto Алматы,Юрлицо,1.0,31929.00000,31929.00000,г.Алматы,Легковые автомобили,D класс,2.0,DE,2019-07-01
4,2019,7,Mercur Auto,Audi,A4,2018,германия,p,1.4,AКПП,FWD,Алматы,Mercur Auto Алматы,Физлицо,1.0,31929.00000,31929.00000,г.Алматы,Легковые автомобили,D класс,2.0,DE,2019-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39961,2019,9,ТК КАМАЗ,KAMAZ,58815Z,2019,казахстан,d,2.5,МКПП,RWD,Актобе,ЗапКазКАМАЗ -Актобе,Юрлицо,1.0,48756.92071,48756.92071,Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики,12.0,KZ,2019-09-01
39962,2019,9,ТК КАМАЗ,KAMAZ,58815Z,2019,казахстан,d,2.5,МКПП,RWD,Актобе,ЗапКазКАМАЗ -Актобе,Юрлицо,1.0,48756.92071,48756.92071,Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики,12.0,KZ,2019-09-01
39963,2019,9,ТК КАМАЗ,KAMAZ,58815Z,2019,казахстан,d,2.5,МКПП,RWD,Актобе,ЗапКазКАМАЗ -Актобе,Юрлицо,1.0,48756.92071,48756.92071,Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики,12.0,KZ,2019-09-01
39964,2019,9,ТК КАМАЗ,KAMAZ,58815Z,2019,казахстан,d,2.5,МКПП,RWD,Актобе,ЗапКазКАМАЗ -Актобе,Юрлицо,1.0,48756.92071,48756.92071,Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики,12.0,KZ,2019-09-01


### Округлим данные в столбцах 'цена_usd', 'продажа_usd' до целых значений

In [ ]:
data[['цена_usd', 'продажа_usd']] = data[['цена_usd', 'продажа_usd']].astype(int)

### Изменим порядок столбцов

In [ ]:
second_cols = ['год', 'месяц', 'дата', 'компания', 'бренд', 'модель', 'год_выпуска',
       'страна_производитель', 'код_страны', 'вид_топлива', 'объём_двиг_л',
       'класс_двигателя', 'коробка_передач', 'тип_привода', 'регион', 'область',
       'наименование_дилерского_центра', 'тип_клиента', 'количество',
       'цена_usd', 'продажа_usd', 'сегментация_2013', 'класс_2013']

In [ ]:
#change order of columns by name
data = data[second_cols]

### Данные обработаны и мы можем провести исследование и анализ.   
Мы выгрузим данные в форматах .csv и .xlsx в том числе для их чтения в редакторе Excel.

Для удобства пользования файлом .xlsx после выгрузки осуществим сортировку данных в следующем порядке:
- год
- месяц
- компания
- бренд
- модель
- год_выпуска

In [ ]:
data.columns

Index(['год', 'месяц', 'дата', 'компания', 'бренд', 'модель', 'год_выпуска',
       'страна_производитель', 'код_страны', 'вид_топлива', 'объём_двиг_л',
       'класс_двигателя', 'коробка_передач', 'тип_привода', 'регион',
       'область', 'наименование_дилерского_центра', 'тип_клиента',
       'количество', 'цена_usd', 'продажа_usd', 'сегментация_2013',
       'класс_2013'],
      dtype='object')

In [ ]:
data = data.sort_values(['год', 'месяц', 'компания', 'бренд', 'модель', 'год_выпуска']).reset_index(drop = True)

#### Выгрузим файл в папку в формате Excel в корневую папку   
Воспользуемся функцией `to_excel()` библиотеки `openpyxl`.

Вызовем функцию `df_download`для выгрузки файлов в форматах:
- .csv
- .xlsx

In [ ]:
%time df_download(data, "kz_automarket_2019.xlsx", 'kz_automarket_2019.csv')

Выгружен файл  kz_automarket_2019.csv 
Выгружен файл  kz_automarket_2019.xlsx 
CPU times: user 19.7 s, sys: 437 ms, total: 20.2 s
Wall time: 20.9 s
